# KSC: LLM-Driven **no Composite ver** Test Generation (GPT‑4o, Python)
**Repo:** `temporalio/money-transfer-project-template-python`  
**LLM:** OpenAI GPT‑4o  
**Notebook 목적:** *분기 의무 + Def‑Use 체인 + 예외 경로*를 통합한 테스트를 자동 생성·실행·증분합니다.

In [1]:
#@title 3-0) 런타임 & 의존성 준비
import os, sys, subprocess, pathlib, getpass, shutil

# 1) 기본 상수/경로
REPO_URL = "https://github.com/temporalio/money-transfer-project-template-python.git"
PROJECT_NAME = REPO_URL.rstrip("/").split("/")[-1].replace(".git", "")
ROOT = pathlib.Path(".").resolve()
PROJ = ROOT / PROJECT_NAME

def sh(cmd: str, check: bool = True, cwd: pathlib.Path | None = None) -> None:
    print("> ", cmd)
    rc = subprocess.call(cmd, shell=True, cwd=str(cwd) if cwd else None)
    if check and rc != 0:
        raise RuntimeError(f"Command failed (rc={rc}): {cmd}")

print(f"Python {sys.version}")
print("ROOT:", ROOT)

# 2) 레포 클론/업데이트
if PROJ.exists():
    print(f"Repo exists at {PROJ}. Pulling latest…")
    sh(f"git -C {PROJ} fetch --all --prune")
    sh(f"git -C {PROJ} reset --hard origin/main")
else:
    sh(f"git clone --depth=1 {REPO_URL}")
print("✅ 레포 클론/업데이트 완료")

# 3) pip 의존성 설치
def pip_install(pkgs):
    cmd = [sys.executable, "-m", "pip", "install", "-U"] + pkgs
    print("> ", " ".join(cmd))
    subprocess.run(cmd, check=True)

pkgs_core = [
    "pytest==7.4.4", "pytest-asyncio==0.23.7", "pytest-mock==3.11.1", "pytest-cov==4.1.0",
    "coverage[toml]==7.6.1",
    "temporalio==1.7.0",
    "lxml==5.2.2",
    "asttokens==2.4.1", "libcst==1.4.0", "networkx==3.3",
    "rich==13.7.1", "pyyaml==6.0.2",
]
if sys.version_info < (3,11):
    pkgs_core.append("tomli")

pkgs_llm = [
    "openai==1.43.0",
    "httpx==0.27.2",
    "backoff==2.2.1",
    "tiktoken==0.7.0",
    "aiolimiter==1.1.0",
    "anyio==4.4.0",
    "nest_asyncio==1.6.0",
    "python-dotenv==1.0.1",
    "tqdm==4.66.5",
]

pip_install(["pip"])
pip_install(pkgs_core)
pip_install(pkgs_llm)
print("✅ 의존성 설치 완료")

# 4) .env 생성
try:
    from dotenv import load_dotenv
except:
    pip_install(["python-dotenv>=1.0.1"])
    from dotenv import load_dotenv

openai_api_key = getpass.getpass("Enter OPENAI_API_KEY (필수): ").strip()
env_path = ROOT / ".env"
env_path.write_text(f"OPENAI_API_KEY={openai_api_key}\n", encoding="utf-8")
load_dotenv(env_path)
os.environ["OPENAI_API_KEY"] = openai_api_key
print("✅ .env 저장 및 환경 주입 완료")

# 5) PYTHONPATH 설정
os.environ["PROJECT_PATH"] = str(PROJ)
py_paths = [str(PROJ)]
if (PROJ / "src").exists():
    py_paths.insert(0, str(PROJ / "src"))
prev_pp = os.environ.get("PYTHONPATH","")
os.environ["PYTHONPATH"] = ":".join(py_paths + ([prev_pp] if prev_pp else []))

print("✅ PYTHONPATH =", os.environ["PYTHONPATH"])

# 6) A안: coverage source 전체(.) 고정하여 .coveragerc 생성
coveragerc_text = """
[run]
branch = True
source =
    .
omit =
    tests/*
    generated_tests/*
    run_artifacts/*
    htmlcov/*
    .venv/*
    */site-packages/*

[report]
show_missing = True
skip_covered = True
"""
(PROJ / ".coveragerc").write_text(coveragerc_text.strip() + "\n", encoding="utf-8")
os.environ["COVERAGE_RCFILE"] = str(PROJ / ".coveragerc")

print("✅ .coveragerc 생성 완료")
print("  - source = . (프로젝트 전체)")
print("  - omit   = tests/, generated_tests/, run_artifacts/, htmlcov/, .venv, site-packages")

# 7) 결과 디렉터리 준비
for d in ["run_artifacts", "htmlcov", "reports"]:
    (PROJ / d).mkdir(parents=True, exist_ok=True)

print("✅ 결과 디렉토리 준비 완료")


Python 3.12.12 (main, Oct 10 2025, 08:52:57) [GCC 11.4.0]
ROOT: /content
>  git clone --depth=1 https://github.com/temporalio/money-transfer-project-template-python.git
✅ 레포 클론/업데이트 완료
>  /usr/bin/python3 -m pip install -U pip
>  /usr/bin/python3 -m pip install -U pytest==7.4.4 pytest-asyncio==0.23.7 pytest-mock==3.11.1 pytest-cov==4.1.0 coverage[toml]==7.6.1 temporalio==1.7.0 lxml==5.2.2 asttokens==2.4.1 libcst==1.4.0 networkx==3.3 rich==13.7.1 pyyaml==6.0.2
>  /usr/bin/python3 -m pip install -U openai==1.43.0 httpx==0.27.2 backoff==2.2.1 tiktoken==0.7.0 aiolimiter==1.1.0 anyio==4.4.0 nest_asyncio==1.6.0 python-dotenv==1.0.1 tqdm==4.66.5
✅ 의존성 설치 완료
Enter OPENAI_API_KEY (필수): ··········
✅ .env 저장 및 환경 주입 완료
✅ PYTHONPATH = /content/money-transfer-project-template-python:/env/python
✅ .coveragerc 생성 완료
  - source = . (프로젝트 전체)
  - omit   = tests/, generated_tests/, run_artifacts/, htmlcov/, .venv, site-packages
✅ 결과 디렉토리 준비 완료


In [2]:
#@title 3-1) 기준선 측정 (실행 가능한 라인/브랜치 기반)

import os, sys, json, subprocess, shutil, re
from pathlib import Path
from lxml import etree

# ====== 0) 경로/환경 ======
assert "PROJ" in globals(), "3-0 단계가 먼저 실행되어야 합니다."
PROJ = Path(PROJ).resolve()
ART_DIR = PROJ / "run_artifacts" / "run1"
HTML_DIR = PROJ / "htmlcov"
ART_DIR.mkdir(parents=True, exist_ok=True)
HTML_DIR.mkdir(parents=True, exist_ok=True)

RCFILE = PROJ / ".coveragerc"
rc_opt = f" --rcfile {RCFILE}" if RCFILE.exists() else ""

def sh(cmd, check=False):
    print("> ", cmd)
    rc = subprocess.call(cmd, shell=True, cwd=str(PROJ))
    if check and rc != 0:
        raise RuntimeError(f"Command failed (rc={rc}): {cmd}")
    return rc

def norm(fp):
    p = Path(fp)
    if not p.is_absolute():
        p = (PROJ / p).resolve()
    return str(p)

def in_proj(abs_path):
    return str(PROJ) in str(Path(abs_path).resolve())

# ====== 1) baseline 테스트 실행 ======
print("🧪 BASELINE 테스트 실행 중…")
sh("coverage erase" + rc_opt)
sh(f"{sys.executable} -m coverage run{rc_opt} -m pytest -q")
sh(f"coverage json -o coverage_base.json{rc_opt}")
sh(f"coverage xml  -o coverage_base.xml{rc_opt}")
sh("coverage html" + rc_opt)

# 복사
json_path = PROJ / "coverage_base.json"
xml_path  = PROJ / "coverage_base.xml"
shutil.copy2(json_path, ART_DIR / "coverage_base.json")
shutil.copy2(xml_path, ART_DIR / "coverage_base.xml")

# ====== 2) baseline JSON 파싱 ======
cov = json.loads(json_path.read_text(encoding="utf-8"))
files = cov.get("files", {}) or {}

# 실행 가능 라인 계산 = executed + missing
def sum_len(key):
    return sum(len((files.get(f, {}) or {}).get(key, []) or []) for f in files)

base_exec = sum_len("executed_lines")
base_miss = sum_len("missing_lines")
total_executable_lines = base_exec + base_miss

# ====== 3) branch coverage (coverage XML → arc 기반) ======
full = half = zero = 0
observed_outcomes = {}

xml_root = etree.parse(str(xml_path)).getroot()
for cls in xml_root.findall(".//class"):
    filename = cls.get("filename")
    abs_f = norm(filename)
    if not in_proj(abs_f):
        continue

    for line in cls.findall("./lines/line"):
        if line.get("branch") != "true":
            continue

        num = int(line.get("number"))
        cond = line.get("condition-coverage")
        m = re.search(r"\((\d+)\s*/\s*(\d+)\)", cond) if cond else None
        if not m:
            continue

        covered, total = int(m.group(1)), int(m.group(2))
        observed_outcomes.setdefault(abs_f, {})[num] = {
            "covered": covered,
            "total": total,
            "ratio": round(covered / total, 3)
        }

        if covered == 0:
            zero += 1
        elif covered == total:
            full += 1
        else:
            half += 1

# 실행 가능한 branch 수
branch_points = full + half + zero

# ====== 4) uncovered_map 생성 ======
uncovered_map = {}
for f, finfo in files.items():
    abs_f = norm(f)
    if not in_proj(abs_f):
        continue
    if any(seg in abs_f for seg in ["tests/", "generated_tests/", "run_artifacts/", "htmlcov/"]):
        continue

    miss = finfo.get("missing_lines", []) or []
    if miss:
        uncovered_map[abs_f] = sorted(set(miss))

(ART_DIR / "uncovered_map_base.json").write_text(
    json.dumps(uncovered_map, indent=2, ensure_ascii=False),
    encoding="utf-8"
)
(ART_DIR / "observed_outcomes_base.json").write_text(
    json.dumps(observed_outcomes, indent=2, ensure_ascii=False),
    encoding="utf-8"
)

# ====== 5) baseline_info 저장 (실행 가능한 라인/브랜치 기준) ======
baseline_info = {
    "total_executable_lines": total_executable_lines,
    "total_executable_branches": branch_points,
    "baseline_executed_lines": base_exec,
    "baseline_missing_lines": base_miss,
    "baseline_branch_full": full,
    "baseline_branch_half": half,
    "baseline_branch_zero": zero,
}

(ART_DIR / "baseline_info.json").write_text(
    json.dumps(baseline_info, indent=2, ensure_ascii=False),
    encoding="utf-8"
)

# ====== 6) 출력 ======
print("✅ 기준선(3-1) 생성 완료")
print(" - uncovered_map_base.json 생성됨")
print(" - observed_outcomes_base.json 생성됨")
print(" - baseline_info.json 생성됨")

print("\n📘 [Baseline Summary]")
print(f" • 전체 실행 가능한 라인수      : {total_executable_lines}")
print(f" • 전체 실행 가능한 분기 수     : {branch_points}")

print("\n📊 [Baseline Coverage]")
print(f" • executed={base_exec}, missing={base_miss}")
pct = round((base_exec / (base_exec + base_miss)) * 100, 2) if (base_exec + base_miss) else 0
print(f"   → 라인 커버리지 비율 : {pct}%")

print("\n🔀 [Baseline Branch Coverage]")
print(f" • full-hit  : {full}")
print(f" • half-hit  : {half}")
print(f" • zero-hit  : {zero}")
pct_b = round((full / branch_points) * 100, 2) if branch_points else 0
print(f"   → 브랜치 커버리지 비율 : {pct_b}%")


🧪 BASELINE 테스트 실행 중…
>  coverage erase --rcfile /content/money-transfer-project-template-python/.coveragerc
>  /usr/bin/python3 -m coverage run --rcfile /content/money-transfer-project-template-python/.coveragerc -m pytest -q
>  coverage json -o coverage_base.json --rcfile /content/money-transfer-project-template-python/.coveragerc
>  coverage xml  -o coverage_base.xml --rcfile /content/money-transfer-project-template-python/.coveragerc
>  coverage html --rcfile /content/money-transfer-project-template-python/.coveragerc
✅ 기준선(3-1) 생성 완료
 - uncovered_map_base.json 생성됨
 - observed_outcomes_base.json 생성됨
 - baseline_info.json 생성됨

📘 [Baseline Summary]
 • 전체 실행 가능한 라인수      : 156
 • 전체 실행 가능한 분기 수     : 20

📊 [Baseline Coverage]
 • executed=102, missing=54
   → 라인 커버리지 비율 : 65.38%

🔀 [Baseline Branch Coverage]
 • full-hit  : 16
 • half-hit  : 0
 • zero-hit  : 4
   → 브랜치 커버리지 비율 : 80.0%


In [3]:
#@title 3-2a) 기존 테스트 프루닝 – 테스트 함수 단위 (기준선 대비 고유 Δcoverage + 탐욕적 선택; 수집필터/폴백 강화)
import subprocess, json, os, shutil, re, sys
from pathlib import Path

# ---------- 0. 경로/전제 ----------
assert 'PROJ' in globals(), "3-0 단계를 먼저 실행해야 합니다."
PROJ = Path(PROJ).resolve()
ART_DIR = PROJ / "run_artifacts" / "run1"
SRC_TESTS = PROJ / "tests"
DST_PRUNED = PROJ / "Pruned_Base_Tests"
DST_PRUNED.mkdir(exist_ok=True)

RCFILE = PROJ / ".coveragerc"
assert RCFILE.exists(), ".coveragerc가 없습니다. 3-0 단계를 먼저 실행하세요."
rc_opt = f"--rcfile={RCFILE}"

BASE_JSON = ART_DIR / "coverage_base.json"
assert BASE_JSON.exists(), "기준선 coverage_base.json이 없습니다. 3-1 단계를 먼저 실행하세요."

# ---------- 1. 기준선 라인 집합 ----------
def load_executed_lines_from_json(json_path: Path) -> set[str]:
    if not json_path.exists():
        return set()
    data = json.loads(json_path.read_text(encoding="utf-8"))
    lines = set()
    for f, finfo in (data.get("files", {}) or {}).items():
        for ln in (finfo.get("executed_lines", []) or []):
            lines.add(f"{f}:{ln}")
    return lines

baseline_lines = load_executed_lines_from_json(BASE_JSON)
print(f"📊 기준선 라인 수: {len(baseline_lines)}")

# ---------- 2. 테스트 목록 수집 (nodeid) ----------
print("📋 pytest 테스트 수집 중 (--collect-only)...")
collect_out = subprocess.check_output(
    [sys.executable, "-m", "pytest", "--collect-only", "-q"],
    cwd=PROJ,
    stderr=subprocess.STDOUT,
).decode("utf-8", errors="ignore")

# nodeid로 간주: "path::..." 패턴만 허용 (안내문/총계 라인 필터)
nodeid_pat = re.compile(r".+::.+")
test_items = [ln.strip() for ln in collect_out.splitlines()
              if nodeid_pat.match(ln.strip())]
print(f"총 수집된 테스트 수: {len(test_items)}")

if not test_items:
    print("ℹ️ 수집된 테스트가 없습니다. 프루닝을 건너뜁니다.")
    (ART_DIR / "pruning_summary.json").write_text(
        json.dumps({
            "total_tests": 0, "retained": 0, "removed": 0,
            "retained_tests": [], "removed_tests": [],
        }, indent=2, ensure_ascii=False),
        encoding="utf-8"
    )
else:
    # ---------- 3. 개별 테스트 실행 → 실행 라인 집합 ----------
    def get_executed_lines_tmp() -> set[str]:
        tmp_json = PROJ / "coverage_tmp.json"
        return load_executed_lines_from_json(tmp_json)

    def run_test_item(item: str) -> tuple[bool, set[str]]:
        subprocess.run(f"coverage erase {rc_opt}", shell=True, cwd=PROJ)
        rc = subprocess.call(
            f"{sys.executable} -m coverage run {rc_opt} -m pytest -q {item}",
            shell=True, cwd=PROJ,
        )
        subprocess.run(f"coverage json -o coverage_tmp.json {rc_opt}", shell=True, cwd=PROJ)
        return (rc == 0), get_executed_lines_tmp()

    per_test_exec: list[tuple[str, bool, set[str]]] = []
    for i, item in enumerate(test_items, 1):
        print(f"[{i}/{len(test_items)}] ▶ 실행 중: {item}")
        ok, exec_set = run_test_item(item)
        per_test_exec.append((item, ok, exec_set))
        print(f"   → {'✅PASS' if ok else '❌FAIL'}, executed_lines={len(exec_set)}")

    # ---------- 4. 탐욕적 선택 (set-cover 유사) ----------
    covered_so_far = set(baseline_lines)
    candidates = [(it, s) for (it, ok, s) in per_test_exec if ok]

    THRESHOLD_MIN_GAIN = 1  # 고유 기여 라인 ≥ 1
    selected: list[str] = []

    while True:
        best = None
        best_gain_val = 0
        for it, s in candidates:
            gain_val = len(s - covered_so_far)
            if gain_val > best_gain_val:
                best_gain_val = gain_val
                best = (it, s)
        if not best or best_gain_val < THRESHOLD_MIN_GAIN:
            break
        it, s = best
        selected.append(it)
        covered_so_far |= s
        candidates = [(iit, ss) for (iit, ss) in candidates if iit != it]

    # ---- 폴백: 모두 0이면 PASS 중에서 최대 executed_lines 1개 유지
    if not selected:
        passables = [(it, s) for (it, ok, s) in per_test_exec if ok]
        if passables:
            it, s = max(passables, key=lambda x: len(x[1]))
            selected = [it]
            covered_so_far |= s
            print(f"ℹ️ 폴백 적용: '{it}' 1개 유지 (실행 라인 {len(s)}).")

    # ---------- 5. 결과 요약 ----------
    selected_set = set(selected)
    retained = [{"name": it, "success": True} for it in selected]
    removed = [{"name": it, "success": bool(ok)}
               for (it, ok, _s) in per_test_exec if it not in selected_set]

    summary = {
        "total_tests": len(test_items),
        "retained": len(retained),
        "removed": len(removed),
        "retained_tests": retained,
        "removed_tests": removed,
        "baseline_lines": len(baseline_lines),
        "final_covered_lines": len(covered_so_far),
        "incremental_gain_lines": len(covered_so_far - baseline_lines),
        "threshold_min_gain": THRESHOLD_MIN_GAIN,
    }
    (ART_DIR / "pruning_summary.json").write_text(
        json.dumps(summary, indent=2, ensure_ascii=False),
        encoding="utf-8"
    )

    # ---------- 6. 유지된 테스트만 복사 ----------
    retained_files = set()
    for nodeid in selected:
        file_part = nodeid.split("::", 1)[0]
        path_obj = (PROJ / file_part).resolve()
        if path_obj.exists():
            retained_files.add(path_obj)

    if SRC_TESTS.exists():
        copied = 0
        for test_file in sorted(SRC_TESTS.rglob("test_*.py")):
            if test_file.resolve() in retained_files:
                rel = test_file.relative_to(SRC_TESTS)
                dst = DST_PRUNED / rel
                dst.parent.mkdir(parents=True, exist_ok=True)
                shutil.copy2(test_file, dst)
                copied += 1
        print(f"📦 유지 파일 복사 완료: {copied}개 파일 → {DST_PRUNED}")
    else:
        print("ℹ️ tests/ 폴더가 없어 파일 복사는 건너뜁니다.")

    (ART_DIR / "retained_nodeids.txt").write_text("\n".join(selected), encoding="utf-8")

    print("✅ 테스트 함수 단위 프루닝 완료")
    print(" - pruning_summary.json  :", ART_DIR / "pruning_summary.json")
    print(" - retained_nodeids.txt  :", ART_DIR / "retained_nodeids.txt")
    print(" - 출력 디렉터리(선택 파일):", DST_PRUNED)


📊 기준선 라인 수: 102
📋 pytest 테스트 수집 중 (--collect-only)...
총 수집된 테스트 수: 3
[1/3] ▶ 실행 중: tests/test_run_worker.py::test_money_transfer
   → ✅PASS, executed_lines=91
[2/3] ▶ 실행 중: tests/test_run_worker.py::test_money_transfer_withdraw_insufficient_funds
   → ✅PASS, executed_lines=83
[3/3] ▶ 실행 중: tests/test_run_worker.py::test_money_transfer_withdraw_invalid_account
   → ✅PASS, executed_lines=79
ℹ️ 폴백 적용: 'tests/test_run_worker.py::test_money_transfer' 1개 유지 (실행 라인 91).
📦 유지 파일 복사 완료: 1개 파일 → /content/money-transfer-project-template-python/Pruned_Base_Tests
✅ 테스트 함수 단위 프루닝 완료
 - pruning_summary.json  : /content/money-transfer-project-template-python/run_artifacts/run1/pruning_summary.json
 - retained_nodeids.txt  : /content/money-transfer-project-template-python/run_artifacts/run1/retained_nodeids.txt
 - 출력 디렉터리(선택 파일): /content/money-transfer-project-template-python/Pruned_Base_Tests


In [ ]:
#@title 3-2b) 복합 목표 생성
import ast
import json
from pathlib import Path
from typing import Dict, List, Tuple, Set, Optional

# ---------- 경로 ----------
PROJ_PATH = Path(PROJ).resolve()
ART_DIR    = PROJ_PATH / "run_artifacts" / "run1"
UNCOVERED_JSON = ART_DIR / "uncovered_map_base.json"
OBSERVED_JSON  = ART_DIR / "observed_outcomes_base.json"

assert UNCOVERED_JSON.exists(),  "uncovered_map_base.json이 없습니다. 3-1을 먼저 실행하세요."
assert OBSERVED_JSON.exists(),   "observed_outcomes_base.json이 없습니다. 3-1을 먼저 실행하세요."

# ===================== 유틸 =====================
def norm_abs(p: str) -> str:
    q = Path(p)
    if not q.is_absolute():
        q = (PROJ_PATH / q).resolve()
    return str(q.resolve())

def rel_from_proj(abs_path: str) -> str:
    try:
        return str(Path(abs_path).resolve().relative_to(PROJ_PATH))
    except Exception:
        return abs_path

def is_source(abs_path: str) -> bool:
    try:
        rel = Path(abs_path).resolve().relative_to(PROJ_PATH)
    except Exception:
        return False
    s = str(rel).replace("\\", "/")
    if not s.endswith(".py"):
        return False
    if s.startswith(("generated_tests/", "tests/", ".venv/", "venv/", "run_artifacts/", "htmlcov/")):
        return False
    return True

def try_unparse(node: ast.AST) -> Optional[str]:
    try:
        if hasattr(ast, "unparse"):
            return ast.unparse(node)
    except Exception:
        return None


# ===================== baseline 입력 로드 =====================
raw_uncovered = json.loads(UNCOVERED_JSON.read_text(encoding="utf-8"))
uncovered_map: Dict[str, List[int]] = {
    norm_abs(k): sorted(set(v))
    for k, v in raw_uncovered.items()
    if is_source(norm_abs(k))
}

raw_observed = json.loads(OBSERVED_JSON.read_text(encoding="utf-8"))
observed_outcomes: Dict[str, Dict[int, Dict[str, float]]] = {}

for k, mp in raw_observed.items():
    abs_k = norm_abs(k)
    if not is_source(abs_k):
        continue
    fixed = {}
    for ln_str, meta in mp.items():
        try:
            fixed[int(ln_str)] = meta
        except:
            continue
    observed_outcomes[abs_k] = fixed

# half-hit
half_hit_map: Dict[str, Set[int]] = {}
for fp, mapping in observed_outcomes.items():
    half = {ln for ln, meta in mapping.items()
            if 0 < int(meta.get("covered", 0)) < int(meta.get("total", 0))}
    if half:
        half_hit_map[fp] = half


# ===================== AST 스캔 =====================
MOD_REQS = {"requests"}
MOD_HTTPX = {"httpx"}
MOD_TEMPORAL = {"temporalio"}
MOD_SUBPROCESS = {"subprocess"}
TEMPORAL_PREFIXES = ("temporalio.client.", "temporalio.worker.", "temporalio.workflow.")

class FunctionInfo:
    def __init__(self, name, lineno):
        self.name = name or "<module>"
        self.lineno = lineno
        self.branches: List[int] = []
        self.defs: Dict[str, List[int]] = {}
        self.uses: Dict[str, List[int]] = {}
        self.exceptions: List[Tuple[str, int, dict]] = []
        self.side_effect_calls: List[Tuple[str, int]] = []

    def add_def(self, v, ln): self.defs.setdefault(v, []).append(ln)
    def add_use(self, v, ln): self.uses.setdefault(v, []).append(ln)


class ASTVisitor(ast.NodeVisitor):
    def __init__(self, file_rel):
        self.file_rel = file_rel
        self.stack: List[FunctionInfo] = []
        self.funcs: List[FunctionInfo] = []
        self.alias_to_module = {}
        self.symbol_to_module = {}

    def current(self):
        if not self.stack:
            if not self.funcs or self.funcs[0].name != "<module>":
                fi = FunctionInfo("<module>", 1)
                self.funcs.insert(0, fi)
            return self.funcs[0]
        return self.stack[-1]

    # imports
    def visit_Import(self, node):
        for alias in node.names:
            mod = alias.name
            asname = alias.asname or mod.split(".")[0]
            self.alias_to_module[asname] = mod
        self.generic_visit(node)

    def visit_ImportFrom(self, node):
        mod = node.module or ""
        for alias in node.names:
            self.symbol_to_module[alias.asname or alias.name] = mod
        self.generic_visit(node)

    # function defs
    def visit_FunctionDef(self, node):
        fi = FunctionInfo(node.name, node.lineno)
        self.stack.append(fi); self.generic_visit(node); self.stack.pop()
        self.funcs.append(fi)

    def visit_AsyncFunctionDef(self, node):
        self.visit_FunctionDef(node)

    # branches
    def visit_If(self, node):        self.current().branches.append(node.lineno); self.generic_visit(node)
    def visit_While(self, node):     self.current().branches.append(node.lineno); self.generic_visit(node)
    def visit_For(self, node):       self.current().branches.append(node.lineno); self.generic_visit(node)
    def visit_AsyncFor(self, node):  self.current().branches.append(node.lineno); self.generic_visit(node)
    def visit_With(self, node):      self.current().branches.append(node.lineno); self.generic_visit(node)

    def visit_Try(self, node):
        self.current().branches.append(node.lineno)
        self.current().exceptions.append(("try", node.lineno, {}))
        self.generic_visit(node)

    def visit_ExceptHandler(self, node):
        hint = {}
        if node.type is not None:
            typ = try_unparse(node.type)
            if not typ and hasattr(node.type, "id"):
                typ = node.type.id
            if typ: hint["exception_type"] = typ
        self.current().branches.append(node.lineno)
        self.current().exceptions.append(("except", node.lineno, hint))
        self.generic_visit(node)

    def visit_Raise(self, node):
        hint = {}
        if node.exc is not None:
            text = try_unparse(node.exc)
            if text: hint["expr"] = text
            if isinstance(node.exc, ast.Call):
                fn = node.exc.func
                if isinstance(fn, ast.Name):
                    hint["exception_type"] = fn.id
                elif isinstance(fn, ast.Attribute):
                    hint["exception_type"] = fn.attr
                if node.exc.args:
                    a0 = node.exc.args[0]
                    if isinstance(a0, ast.Constant) and isinstance(a0.value, str):
                        hint["message_contains"] = a0.value[:80]
        self.current().exceptions.append(("raise", node.lineno, hint))
        self.generic_visit(node)

    def visit_Assert(self, node):
        hint = {}
        if node.msg and isinstance(node.msg, ast.Constant) and isinstance(node.msg.value, str):
            hint["message_contains"] = node.msg.value[:80]
        self.current().exceptions.append(("assert", node.lineno, hint))
        self.generic_visit(node)

    def visit_Name(self, node):
        if isinstance(node.ctx, ast.Store):
            self.current().add_def(node.id, node.lineno)
        elif isinstance(node.ctx, ast.Load):
            self.current().add_use(node.id, node.lineno)
        self.generic_visit(node)

    # dotted name
    def _dotted_name(self, node):
        if isinstance(node, ast.Name):
            nm = node.id
            if nm in self.alias_to_module:
                return self.alias_to_module[nm]
            if nm in self.symbol_to_module:
                return f"{self.symbol_to_module[nm]}.{nm}"
            return nm
        if isinstance(node, ast.Attribute):
            base = self._dotted_name(node.value)
            if base: return f"{base}.{node.attr}"
        return None

    # calls
    def visit_Call(self, node):
        qn = self._dotted_name(node.func) or ""
        if qn in ("open", "builtins.open"):
            self.current().side_effect_calls.append(("io_open", node.lineno))
        if any(qn.startswith(m + ".") for m in MOD_REQS):
            self.current().side_effect_calls.append(("net_requests", node.lineno))
        if any(qn.startswith(m + ".") for m in MOD_HTTPX):
            self.current().side_effect_calls.append(("net_httpx", node.lineno))
        if any(qn.startswith(pref) for pref in TEMPORAL_PREFIXES) \
           or any(qn.startswith(m + ".") for m in MOD_TEMPORAL):
            self.current().side_effect_calls.append(("temporal", node.lineno))
        if any(qn.startswith(m + ".") for m in MOD_SUBPROCESS):
            self.current().side_effect_calls.append(("subprocess", node.lineno))
        self.generic_visit(node)


# ===================== 대상 파일 =====================
candidate_files = sorted(set([*uncovered_map.keys(), *observed_outcomes.keys()]))
candidate_files = [fp for fp in candidate_files if is_source(fp)]


# ---------------- 가중치/하이퍼파라미터 ----------------
ALPHA_BRANCH = 0.45
BETA_DU      = 0.25
GAMMA_EXC    = 0.30
LAMBDA_1_CONST   = 0.20
LAMBDA_2_CONTEXT = 0.60
LAMBDA_3_TARGETS = 0.20
TOP_K = 8
OVERLAP_THETA = 0.50

assert abs((ALPHA_BRANCH + BETA_DU + GAMMA_EXC) - 1.0) < 1e-6
assert abs((LAMBDA_1_CONST + LAMBDA_2_CONTEXT + LAMBDA_3_TARGETS) - 1.0) < 1e-6


def coverage_gain_structural(b_cnt, du_cnt, exc_cnt):
    return ALPHA_BRANCH*b_cnt + BETA_DU*du_cnt + GAMMA_EXC*exc_cnt

def coverage_gain_total(target_lines, b_cnt, du_cnt, exc_cnt):
    return len(target_lines) + coverage_gain_structural(b_cnt, du_cnt, exc_cnt)

def generation_cost(context_size, target_count):
    return (LAMBDA_1_CONST*1.0) + (LAMBDA_2_CONTEXT*context_size) + (LAMBDA_3_TARGETS*target_count)


# ===================== AST → file_infos =====================
file_infos = {}
MAX_NEAR_GAP = 8

for file_abs in candidate_files:
    try:
        source = Path(file_abs).read_text(encoding="utf-8")
        tree = ast.parse(source)
    except Exception as e:
        print(f"⚠️ AST parse 실패: {file_abs}, {e}")
        continue

    rel = rel_from_proj(file_abs)
    visitor = ASTVisitor(rel)
    visitor.visit(tree)

    miss = set(uncovered_map.get(file_abs, []))
    half = set(half_hit_map.get(file_abs, []))

    for fi in visitor.funcs:
        tb = [ln for ln in fi.branches if (ln in miss or ln in half)]
        tu_pairs = []
        for var, defs in fi.defs.items():
            uses = sorted(fi.uses.get(var, []))
            defs_sorted = sorted(defs)
            for u in uses:
                if u not in miss:
                    continue
                d_le = [d for d in defs_sorted if d <= u]
                if not d_le:
                    continue
                d = max(d_le)
                tu_pairs.append((var, d, u))

        te = [(kind, line, hint)
              for (kind, line, hint) in fi.exceptions if line in miss]

        ctx = len({ln for ls in fi.defs.values() for ln in ls} |
                  {ln for ls in fi.uses.values() for ln in ls})

        if tb or tu_pairs or te:
            file_infos[(rel, fi.name)] = {
                "branches": sorted(tb),
                "def_uses": sorted(tu_pairs, key=lambda x: (x[2], x[1], x[0])),
                "exceptions": sorted(te, key=lambda x: (x[1], x[0])),
                "func_lineno": fi.lineno,
                "context_size": ctx,
            }


# ===================== 후보 goal 생성 =====================
def make_goal(file_rel, func, func_lineno, branches, def_uses, exceptions, context_size):
    target_lines = set(branches)
    for _, d, u in def_uses: target_lines.update([d, u])
    for _, line, _ in exceptions: target_lines.add(line)

    b_cnt = len(branches)
    du_cnt = len(def_uses)
    exc_cnt = len(exceptions)

    gain = coverage_gain_total(target_lines, b_cnt, du_cnt, exc_cnt)
    cost = generation_cost(context_size, len(target_lines))

    return {
        "file": file_rel,
        "function": {"name": func, "lineno": func_lineno},
        "components": {
            "branches": [{"line": b} for b in branches],
            "def_uses": [{"var": v, "def_line": d, "use_line": u} for (v, d, u) in def_uses],
            "exceptions": [{"kind": k, "line": ln, **hint} for (k, ln, hint) in exceptions],
        },
        "target_lines": sorted(target_lines),
        "coverage_gain": float(gain),
        "generation_cost": float(cost),
        "context_size": context_size,
    }


candidates = []

for (file_rel, func), info in file_infos.items():
    tb = info["branches"]
    tu = info["def_uses"]
    te = info["exceptions"]
    ctx = info["context_size"]
    func_lineno = info["func_lineno"]

    # 단일 요소
    for b in tb:              candidates.append(make_goal(file_rel, func, func_lineno, [b], [], [], ctx))
    for (v, d, u) in tu:      candidates.append(make_goal(file_rel, func, func_lineno, [], [(v, d, u)], [], ctx))
    for (k, ln, h) in te:     candidates.append(make_goal(file_rel, func, func_lineno, [], [], [(k, ln, h)], ctx))

    # 근접 조합
    for b in tb:
        for (v, d, u) in tu:
            if max(b, u) - min(b, d) <= MAX_NEAR_GAP:
                candidates.append(make_goal(file_rel, func, func_lineno, [b], [(v, d, u)], [], ctx))

    for b in tb:
        for (k, ln, h) in te:
            if abs(b - ln) <= MAX_NEAR_GAP:
                candidates.append(make_goal(file_rel, func, func_lineno, [b], [], [(k, ln, h)], ctx))

    for (v, d, u) in tu:
        for (k, ln, h) in te:
            if max(u, ln) - min(d, ln) <= MAX_NEAR_GAP:
                candidates.append(make_goal(file_rel, func, func_lineno, [], [(v, d, u)], [(k, ln, h)], ctx))

    for b in tb:
        for (v, d, u) in tu:
            for (k, ln, h) in te:
                lines = [b, d, u, ln]
                if max(lines) - min(lines) <= MAX_NEAR_GAP:
                    candidates.append(make_goal(file_rel, func, func_lineno,
                                                [b], [(v, d, u)], [(k, ln, h)], ctx))


# ===================== Algorithm 1 (19줄) 구현 =====================

def overlap_basic(g, h):
    tg = set(g["target_lines"])
    th = set(h["target_lines"])
    if not tg:
        return 0.0
    return len(tg & th) / len(tg)

# goc 계산
for g in candidates:
    gain = g["coverage_gain"]
    cost = g["generation_cost"]
    g["goc"] = gain / cost if cost > 0 else 0.0

# ---- filtering (line 12~16) ----
filtered = []
for g in candidates:
    if g["goc"] <= 0:
        continue
    bad = False
    for h in filtered:
        if overlap_basic(g, h) > OVERLAP_THETA and h["goc"] >= g["goc"]:
            bad = True
            break
    if not bad:
        filtered.append(g)

# ---- 정렬 (line 17) ----
filtered_sorted = sorted(filtered, key=lambda x: x["goc"], reverse=True)

# ---- top-k 선택 (line 18) ----
selected = filtered_sorted[:TOP_K]

# ---- 정제 및 저장 ----
for i, g in enumerate(selected, 1):
    g["id"] = f"{i:04d}"
    g["coverage_gain"] = round(g["coverage_gain"], 6)
    g["generation_cost"] = round(g["generation_cost"], 6)
    g["score"] = round(g["goc"], 6)

(ART_DIR / "goals_raw.json").write_text(json.dumps(selected, indent=2, ensure_ascii=False), encoding="utf-8")

ranked = [
    {
        "id": g["id"],
        "file": g["file"],
        "function": g["function"],
        "components": g["components"],
        "target_lines": g["target_lines"],
        "coverage_gain": g["coverage_gain"],
        "generation_cost": g["generation_cost"],
        "score": g["score"],
        "context_size": g["context_size"],
    }
    for g in selected
]

(ART_DIR / "goals_ranked.json").write_text(json.dumps(ranked, indent=2, ensure_ascii=False), encoding="utf-8")

print(f"✅ 복합 목표 생성 완료 (Algorithm1, top-k={TOP_K})")
print(f" - 원본 후보 수: {len(candidates)}")
print(f" - filtered 후보 수: {len(filtered)}")
print(f" - 최종 선택된 목표 수: {len(selected)}")
print(" - 저장: goals_raw.json, goals_ranked.json")


✅ 복합 목표 생성 완료 (Algorithm1, top-k=8)
 - 원본 후보 수: 94
 - filtered 후보 수: 50
 - 최종 선택된 목표 수: 8
 - 저장: goals_raw.json, goals_ranked.json


In [4]:
#@title 3-2c) 미커버 라인/브랜치 기반 경량 입력 제약 추출기 (복합목표 제거 버전)

import json
from pathlib import Path

PROJ_PATH = Path(PROJ).resolve()
ART_DIR = PROJ_PATH / "run_artifacts" / "run1"

UNCOVERED = ART_DIR / "uncovered_map_base.json"
BRANCHES = ART_DIR / "observed_outcomes_base.json"

assert UNCOVERED.exists(), "uncovered_map_base.json 없음. 3-1 먼저 실행하세요."
assert BRANCHES.exists(), "observed_outcomes_base.json 없음. 3-1 먼저 실행하세요."

out_file = ART_DIR / "goals_minimal.json"

# ------------------------------
# 미커버 라인 + 미커버 브랜치 기반 목표 생성
# ------------------------------

uncovered = json.loads(UNCOVERED.read_text(encoding="utf-8"))
branch_info = json.loads(BRANCHES.read_text(encoding="utf-8"))

goals_out = []

for file_path, miss_lines in uncovered.items():

    file_item = {
        "file": file_path,
        "target_lines": miss_lines,
        "branch_targets": [],
        "input_constraints": []   # 최소 제약 구성
    }

    # ----------- 미커버 브랜치 정보 추가 -----------
    if file_path in branch_info:
        for ln, info in branch_info[file_path].items():
            if info["covered"] == 0:  # 명확한 미커버 브랜치
                file_item["branch_targets"].append({
                    "line": ln,
                    "total_outcomes": info["total"],
                    "covered": info["covered"]
                })

    # ----------- 단순 제약 구성 -----------
    # LLM에게 "이 라인을 커버하는 테스트 만들어라" 식의 최소 constraint
    constraints = []

    # 라인 기반 제약
    for ln in miss_lines:
        constraints.append(f"hit line {ln}")

    # 브랜치 기반 제약
    for b in file_item["branch_targets"]:
        constraints.append(f"cover branch at line {b['line']} with {b['total_outcomes']} outcomes")

    # 중복 제거
    constraints = list(dict.fromkeys(constraints))

    file_item["input_constraints"] = constraints

    goals_out.append(file_item)


# 저장
out_file.write_text(
    json.dumps(goals_out, ensure_ascii=False, indent=2),
    encoding="utf-8"
)

print("✅ 미커버 기반 최소 목표(goal_minimal) 생성 완료 →", out_file)
print("총 파일 수:", len(goals_out))


✅ 미커버 기반 최소 목표(goal_minimal) 생성 완료 → /content/money-transfer-project-template-python/run_artifacts/run1/goals_minimal.json
총 파일 수: 5


In [5]:
#@title 3-3) LLM 프롬프트 생성 (미커버라인/브랜치 기반 버전)

import json, re
from pathlib import Path

PROJ_PATH = Path(PROJ).resolve()
ART_DIR = PROJ_PATH / "run_artifacts" / "run1"

GOALS_MINIMAL = ART_DIR / "goals_minimal.json"
LLM_OUT = ART_DIR / "llm_prompts.jsonl"

assert GOALS_MINIMAL.exists(), "goals_minimal.json 없음. 3-2c(미니멀) 먼저 실행하세요."

goals = json.loads(GOALS_MINIMAL.read_text(encoding="utf-8"))

# -------------------------
# 유틸 함수
# -------------------------

def to_mod(file):
    s = file.replace("\\", "/")
    if s.startswith("src/"): s = s[4:]
    if s.endswith(".py"): s = s[:-3]
    return s.replace("/", ".")

def suggest_filename(goal_idx, file):
    base = Path(file).stem
    safe = re.sub(r"[^a-zA-Z0-9_]+", "_", base)
    return f"test_gen_goal_{goal_idx}_{safe}.py"

# -------------------------
# 시스템 메시지
# -------------------------

SYSTEM = (
"역할: 당신은 주어진 미커버 라인/브랜치 목표를 타격하는 pytest 테스트 코드를 생성하는 엔진입니다.\n"
"출력: 오직 하나의 JSON 객체만 반환하고 주석/markdown 금지.\n"
"{\n"
'  "filename": "test_*.py",\n'
'  "tests": [ {"name": "test_*", "code": "<pytest code>"} ]\n'
"}\n"
"\n"
"엄격 규칙:\n"
"• importlib.import_module + getattr 로만 심볼 접근 (없으면 skip 가능)\n"
"• 최소 1개 assert 또는 pytest.raises 필수\n"
"• target_lines를 적어도 1개 이상 반드시 커버할 것\n"
"• branch target이 있으면 양 경로 테스트 생성 권장\n"
"• IO/시간/env/net/Temporal 등 외부 호출은 모두 monkeypatch\n"
"• async 대상은 반드시 async 테스트 + await\n"
"• asyncio.run 금지\n"
)

# -------------------------
# 프롬프트 생성
# -------------------------

with LLM_OUT.open("w", encoding="utf-8") as outf:
    for idx, g in enumerate(goals, start=1):

        file = g["file"]
        target_lines = g["target_lines"]
        branch_targets = g.get("branch_targets", [])
        constraints = g.get("input_constraints", [])

        mod = to_mod(file)
        filename = suggest_filename(idx, file)

        USER = {
            "schema_version": "v1_minimal",
            "identifier": {
                "goal_index": idx,
                "file": file,
                "suggested_filename": filename
            },
            "project": {
                "root": str(PROJ_PATH),
                "module": mod
            },
            "goal": {
                "target_lines": target_lines,
                "branch_targets": branch_targets,
                "input_constraints": constraints
            },
            "instructions": [
                "주어진 target_lines를 반드시 타격하는 pytest 테스트 생성",
                "branch_targets가 있으면 해당 분기를 모두 커버",
                "importlib + getattr 기반 동적 임포트 사용",
                "테스트는 최소 1개 assert 또는 pytest.raises 포함",
                "async/Temporal 규칙 준수"
            ]
        }

        rec = {
            "meta": {
                "goal_index": idx,
                "file": file,
                "module": mod,
                "suggested_filename": filename,
                "num_target_lines": len(target_lines),
                "num_branch_targets": len(branch_targets),
                "num_constraints": len(constraints)
            },
            "messages": [
                {"role": "system", "content": SYSTEM},
                {"role": "user", "content": json.dumps(USER, ensure_ascii=False, indent=2)}
            ]
        }

        outf.write(json.dumps(rec, ensure_ascii=False) + "\n")

print("✅ 미커버 기반 LLM 프롬프트 생성 완료 →", LLM_OUT)
print("총 목표:", len(goals))
print("예시 파일명:", suggest_filename(1, goals[0]['file']))


✅ 미커버 기반 LLM 프롬프트 생성 완료 → /content/money-transfer-project-template-python/run_artifacts/run1/llm_prompts.jsonl
총 목표: 5
예시 파일명: test_gen_goal_1_activities.py


In [6]:
#@title 3-4) 테스트 코드 생성 (미커버 목표 기반 버전)

import os
import re
import json
import time
import ast
from pathlib import Path
import httpx
import backoff
from openai import OpenAI, APIError, RateLimitError, APIConnectionError

# ---------- 경로 설정 ----------
ART_DIR = Path(PROJ) / "run_artifacts" / "run1"
LLM_PROMPTS_PATH = ART_DIR / "llm_prompts.jsonl"
GEN_DIR = Path(PROJ) / "generated_tests"
RAW_DIR = ART_DIR / "_raw"
ERR_DIR = ART_DIR / "_errors"
GEN_DIR.mkdir(parents=True, exist_ok=True)
RAW_DIR.mkdir(parents=True, exist_ok=True)
ERR_DIR.mkdir(parents=True, exist_ok=True)

# ---------- OpenAI 클라이언트 ----------
if not os.getenv("OPENAI_API_KEY"):
    raise RuntimeError("OPENAI_API_KEY가 없습니다.")

http_client = httpx.Client(
    timeout=180.0,
    follow_redirects=True,
    limits=httpx.Limits(max_connections=1, max_keepalive_connections=0),
    transport=httpx.HTTPTransport(retries=5),
)
client = OpenAI(api_key=os.getenv("OPENAI_API_KEY"), http_client=http_client)

# ---------- 유틸 ----------
_slug_re = re.compile(r"[^a-z0-9_]+")

def slugify(s: str, maxlen: int = 40) -> str:
    s = s.lower().strip().replace("-", "_").replace(" ", "_")
    s = _slug_re.sub("_", s)
    s = re.sub(r"_+", "_", s).strip("_")
    return s[:maxlen] or "t"

def strip_fences(s: str) -> str:
    s = re.sub(r"^```[a-zA-Z0-9]*\s*", "", s.strip())
    s = re.sub(r"\s*```$", "", s)
    return s

def ensure_unique_path(base: Path) -> Path:
    p = base
    i = 2
    while p.exists():
        p = base.with_name(f"{base.stem}_{i}{base.suffix}")
        i += 1
    return p

def write_error(goal_idx: int, kind: str, payload: dict, idx: int | None = None):
    tag = f"goal_{goal_idx}_{kind}" if idx is None else f"goal_{goal_idx}_t{idx}_{kind}"
    (ERR_DIR / f"{tag}.json").write_text(
        json.dumps(payload, ensure_ascii=False, indent=2), encoding="utf-8"
    )

# ---------- import 보정 ----------
def auto_fix_imports(code: str) -> str:
    lines = code.splitlines()
    text = "\n".join(lines)

    need_pytest = ("pytest." in text) and not re.search(r"^\s*import\s+pytest\b", text, re.M)
    need_importlib = ("importlib." in text) and not re.search(r"^\s*import\s+importlib\b", text, re.M)

    prepend = []
    if need_pytest: prepend.append("import pytest")
    if need_importlib: prepend.append("import importlib")

    if not prepend:
        return code

    return "\n".join(prepend + lines) + "\n"

# ---------- 코드 정리 ----------
def sanitize_test_code(code: str) -> str:
    patterns = [
        re.compile(r"(?ms)^\s*if\s+__name__\s*==\s*['\"]__main__['\"]\s*:\s*\n(?:\s+.*\n?)+$"),
        re.compile(r"(?m)^\s*pytest\.main\s*\(.*?\)\s*$"),
        re.compile(r"(?m)^\s*unittest\.main\s*\(.*?\)\s*$"),
    ]
    new = code
    for pat in patterns:
        new = pat.sub("", new)
    return new.strip() + "\n"

# ---------- AST 검증 ----------
def parse_ast_or_error(code: str):
    try:
        return ast.parse(code), None
    except SyntaxError as e:
        return None, f"syntax_error:{e.msg}@L{e.lineno}"

def extract_test_funcs(tree: ast.AST) -> list:
    return [
        n for n in ast.walk(tree)
        if isinstance(n, ast.FunctionDef) and n.name.startswith("test_")
    ]

def has_assert_or_raises(tree: ast.AST, code: str) -> bool:
    has_assert_stmt = any(isinstance(n, ast.Assert) for n in ast.walk(tree))
    has_pytest_raises = "pytest.raises(" in code
    return has_assert_stmt or has_pytest_raises

# ---------- 최소 검증 ----------
def minimal_viability_checks(code: str):
    reasons = []

    tree, synerr = parse_ast_or_error(code)
    if synerr:
        reasons.append(synerr)
        return False, reasons

    tests = extract_test_funcs(tree)
    if not tests:
        reasons.append("no_test_functions")

    if not has_assert_or_raises(tree, code):
        reasons.append("no_assert_or_raises")

    return (len(reasons) == 0), reasons

# ---------- OpenAI 호출 ----------
@backoff.on_exception(
    backoff.expo,
    (APIConnectionError, APIError, RateLimitError),
    max_tries=8,
    max_time=300
)
def call_openai_with_retry(messages):
    resp = client.chat.completions.create(
        model="gpt-4o",
        messages=messages,
        response_format={"type": "json_object"},
        timeout=200.0,
    )
    return resp.choices[0].message.content

# ---------- filename 결정 ----------
def choose_filename(goal_idx: int, idx: int):
    return f"test_gen_goal_{goal_idx}_{idx}.py"

# ---------- 메인 ----------
gen_log_path = ART_DIR / "gen_log.jsonl"
ok_count = 0
fail_count = 0

with LLM_PROMPTS_PATH.open("r", encoding="utf-8") as f_in, gen_log_path.open("w", encoding="utf-8") as f_log:
    for line in f_in:
        rec = json.loads(line)
        goal_idx = rec["meta"]["goal_index"]

        messages = rec["messages"]
        print(f"\n🚀 Goal {goal_idx} 테스트 생성 요청…")

        try:
            out_text = call_openai_with_retry(messages)
        except Exception as e:
            fail_count += 1
            write_error(goal_idx, "request_error", {"error": str(e)})
            print(f"❌ Goal {goal_idx} 요청 실패: {e}")
            continue

        (RAW_DIR / f"goal_{goal_idx}_raw.json").write_text(out_text, encoding="utf-8")

        try:
            cleaned = strip_fences(out_text)
            result = json.loads(cleaned)
        except Exception as e:
            fail_count += 1
            write_error(goal_idx, "json_parse_error", {"error": str(e), "raw": out_text[:1000]})
            print(f"❌ Goal {goal_idx} JSON 파싱 실패: {e}")
            continue

        tests = result.get("tests", [])
        saved_files = []
        excluded = []

        for idx, t in enumerate(tests, start=1):
            code = sanitize_test_code(strip_fences(t.get("code", "")))
            code = auto_fix_imports(code)

            ok_min, reasons = minimal_viability_checks(code)
            if not ok_min:
                excluded.append({"index": idx, "reasons": reasons})
                write_error(goal_idx, "min_viability", {"index": idx, "reasons": reasons})
                print(f"⚠️ Goal {goal_idx} 테스트 #{idx} 제외: {reasons}")
                continue

            out_filename = choose_filename(goal_idx, idx)
            out_path = ensure_unique_path(GEN_DIR / out_filename)
            out_path.write_text(code, encoding="utf-8")
            saved_files.append(out_path.name)
            print(f"✅ 저장: {out_path.name}")

        if saved_files:
            ok_count += 1
            f_log.write(json.dumps({"goal_idx": goal_idx, "saved_files": saved_files, "excluded": excluded}, ensure_ascii=False) + "\n")
        else:
            fail_count += 1
            write_error(goal_idx, "no_valid_tests", {"excluded": excluded})
            print(f"❌ Goal {goal_idx} 유효 테스트 없음")

print(f"\n✅ 생성 완료: 성공 {ok_count} / 실패 {fail_count}")
print(f"📂 저장 위치: {GEN_DIR}")
print(f"📁 원본: {RAW_DIR}")
print(f"📁 에러: {ERR_DIR}")
print(f"📄 로그: {gen_log_path}")



🚀 Goal 1 테스트 생성 요청…
✅ 저장: test_gen_goal_1_1.py

🚀 Goal 2 테스트 생성 요청…
⚠️ Goal 2 테스트 #1 제외: ['no_test_functions']
❌ Goal 2 유효 테스트 없음

🚀 Goal 3 테스트 생성 요청…
⚠️ Goal 3 테스트 #1 제외: ['no_assert_or_raises']
❌ Goal 3 유효 테스트 없음

🚀 Goal 4 테스트 생성 요청…
✅ 저장: test_gen_goal_4_1.py
✅ 저장: test_gen_goal_4_2.py

🚀 Goal 5 테스트 생성 요청…
⚠️ Goal 5 테스트 #1 제외: ['no_test_functions']
❌ Goal 5 유효 테스트 없음

✅ 생성 완료: 성공 2 / 실패 3
📂 저장 위치: /content/money-transfer-project-template-python/generated_tests
📁 원본: /content/money-transfer-project-template-python/run_artifacts/run1/_raw
📁 에러: /content/money-transfer-project-template-python/run_artifacts/run1/_errors
📄 로그: /content/money-transfer-project-template-python/run_artifacts/run1/gen_log.jsonl


In [7]:
#@title 3-5) 기존 tests(프루닝) + 생성 tests 격리 실행 · 로그 수집 · 샤드 결합 · 향상치 계산 (autosetup 포함)
import os, sys, json, re, time, subprocess, shutil, shlex
from pathlib import Path
from datetime import datetime, timezone
from lxml import etree

# ==== 경로/상수 ====
assert 'PROJ' in globals(), "3-0 단계를 먼저 실행하세요."
PROJ = Path(PROJ).resolve()
ART_DIR = PROJ / "run_artifacts" / "run1"
GEN_DIR = PROJ / "generated_tests"
TESTS_DIR = PROJ / "tests"                   # 원본(백업용)
PRUNED_DIR = PROJ / "Pruned_Base_Tests"      # 프루닝 집합(우선)
LOG_DIR = ART_DIR / "logs"
COV_SHARDS_DIR = ART_DIR / "cov_shards"
HTML_DIR_GEN = PROJ / "htmlcov_gen"

ART_DIR.mkdir(parents=True, exist_ok=True)
LOG_DIR.mkdir(parents=True, exist_ok=True)
COV_SHARDS_DIR.mkdir(parents=True, exist_ok=True)
HTML_DIR_GEN.mkdir(parents=True, exist_ok=True)
GEN_DIR.mkdir(parents=True, exist_ok=True)

RCFILE = PROJ / ".coveragerc"
rc_opt = f" --rcfile {RCFILE}" if RCFILE.exists() else ""

# 실행 파라미터
PY_EXE = sys.executable
TIMEOUT_SEC_GEN = 45            # 생성 테스트 파일 1개당 타임아웃 (늘림)
TIMEOUT_SEC_BASE = 45           # (프루닝된) 기존 테스트 파일 1개당 타임아웃
PYTEST_FLAGS = "-q -s"
ENV_BASE = os.environ.copy()

# goal_id 추출
RE_GOAL = re.compile(r"(?:^|[_-])(?P<gid>\d{4})(?:[_-]|$)")

# ==== Autosetup: pytest.ini + generated_tests/conftest.py ====
def ensure_pytest_ini():
    ini = PROJ / "pytest.ini"
    base = []
    if ini.exists():
        base = ini.read_text(encoding="utf-8").splitlines()

    def set_or_append(lines, key, value):
        if not any(l.strip().startswith("[pytest]") for l in lines):
            lines = ["[pytest]"] + lines
        found = False
        for i, l in enumerate(lines):
            if l.strip().startswith(key):
                lines[i] = f"{key} = {value}"
                found = True
                break
        if not found:
            try:
                idx = next(i for i,l in enumerate(lines) if l.strip().startswith("[pytest]"))
            except StopIteration:
                idx = -1
            insert_at = idx+1 if idx>=0 else len(lines)
            lines.insert(insert_at, f"{key} = {value}")
        return lines

    lines = base[:]
    lines = set_or_append(lines, "asyncio_mode", "auto")  # pytest-asyncio 자동 모드
    if not any(l.strip().startswith("addopts") for l in lines):
        lines.append("addopts = -q -s")
    ini.write_text("\n".join(lines) + "\n", encoding="utf-8")
    print("✅ pytest.ini ensured →", ini)

def ensure_conftest_autouse():
    cf = GEN_DIR / "conftest.py"
    payload = (
        "import os, asyncio, pytest, importlib, inspect\n"
        "\n"
        "@pytest.fixture(autouse=True)\n"
        "def _no_real_net_env(monkeypatch):\n"
        "    monkeypatch.setenv('NO_PROXY', '*')\n"
        "    for k in ['HTTP_PROXY','HTTPS_PROXY','http_proxy','https_proxy','ALL_PROXY','all_proxy']:\n"
        "        monkeypatch.delenv(k, raising=False)\n"
        "\n"
        "@pytest.fixture(autouse=True)\n"
        "def _patch_time(monkeypatch):\n"
        "    import time\n"
        "    monkeypatch.setattr(time, 'sleep', lambda *_: None, raising=False)\n"
        "\n"
        "@pytest.fixture(autouse=True)\n"
        "def _patch_temporal(monkeypatch):\n"
        "    try:\n"
        "        import temporalio  # noqa: F401\n"
        "    except Exception:\n"
        "        return\n"
        "    class _Dummy:\n"
        "        def __init__(self, *a, **k): pass\n"
        "        async def __aenter__(self): return self\n"
        "        async def __aexit__(self, *a): pass\n"
        "        def __call__(self, *a, **k): return self\n"
        "        async def run(self, *a, **k): return None\n"
        "        async def start(self, *a, **k): return None\n"
        "    for path in ['temporalio.client.Client','temporalio.worker.Worker','temporalio.workflow.Workflow']:\n"
        "        try:\n"
        "            mod_name, attr = path.rsplit('.', 1)\n"
        "            mod = importlib.import_module(mod_name)\n"
        "            setattr(mod, attr, _Dummy)\n"
        "        except Exception:\n"
        "            pass\n"
        "\n"
        "def _wrap_async(func):\n"
        "    if inspect.iscoroutinefunction(func):\n"
        "        def sync_wrapper(*a, **k):\n"
        "            try:\n"
        "                loop = asyncio.get_event_loop()\n"
        "            except RuntimeError:\n"
        "                loop = asyncio.new_event_loop()\n"
        "                asyncio.set_event_loop(loop)\n"
        "            return loop.run_until_complete(func(*a, **k))\n"
        "        return sync_wrapper\n"
        "    return func\n"
        "\n"
        "_TARGET_MODULES = [\n"
        "    'run_worker_module',\n"
        "    'run_workflow_module',\n"
        "    'run_workflow_main',\n"
        "]\n"
        "\n"
        "@pytest.fixture(autouse=True)\n"
        "def _wrap_async_entrypoints(monkeypatch):\n"
        "    for mod_name in _TARGET_MODULES:\n"
        "        try:\n"
        "            mod = importlib.import_module(mod_name)\n"
        "        except Exception:\n"
        "            continue\n"
        "        for attr in ('main','run','start'):\n"
        "            if hasattr(mod, attr):\n"
        "                try:\n"
        "                    monkeypatch.setattr(mod, attr, _wrap_async(getattr(mod, attr)), raising=False)\n"
        "                except Exception:\n"
        "                    pass\n"
        "    def _safe_run(coro):\n"
        "        if inspect.iscoroutine(coro):\n"
        "            try:\n"
        "                loop = asyncio.get_event_loop()\n"
        "            except RuntimeError:\n"
        "                loop = asyncio.new_event_loop()\n"
        "                asyncio.set_event_loop(loop)\n"
        "            return loop.run_until_complete(coro)\n"
        "        return coro\n"
        "    monkeypatch.setattr(asyncio, 'run', _safe_run, raising=False)\n"
    )
    if cf.exists():
        text = cf.read_text(encoding="utf-8")
        if "_wrap_async_entrypoints" not in text:
            text = text.rstrip() + "\n\n" + payload
            cf.write_text(text, encoding="utf-8")
            print("✅ conftest.py augmented →", cf)
        else:
            print("✅ conftest.py already has autosetup →", cf)
    else:
        cf.write_text(payload, encoding="utf-8")
        print("✅ conftest.py created →", cf)

ensure_pytest_ini()
ensure_conftest_autouse()

# ==== 유틸 ====
def goal_id_from_name(name: str) -> str | None:
    m = RE_GOAL.search(name)
    return m.group("gid") if m else None

def sh(cmd: str, cwd: Path | None = None, timeout: int | None = None, env: dict | None = None):
    try:
        p = subprocess.run(
            cmd, cwd=str(cwd or PROJ), env=env or ENV_BASE,
            shell=True, stdout=subprocess.PIPE, stderr=subprocess.PIPE,
            timeout=timeout, text=True
        )
        return p.returncode, p.stdout, p.stderr, False
    except subprocess.TimeoutExpired as e:
        return 124, e.stdout or "", e.stderr or "", True

def list_test_files(dir_path: Path) -> list[Path]:
    if not dir_path.exists():
        return []
    files = []
    files += list(dir_path.glob("test*.py"))
    files += list(dir_path.glob("*_test.py"))
    return sorted(set(p for p in files if p.is_file()))

def list_generated_test_files() -> list[Path]:
    if not GEN_DIR.exists():
        return []
    # conftest.py는 수집 대상에서 제외
    return sorted([p for p in GEN_DIR.glob("*.py") if p.is_file() and p.name != "conftest.py"])

def rel_to_proj(p: Path) -> str:
    try:
        return str(p.resolve().relative_to(PROJ))
    except Exception:
        return str(p.resolve())

# ==== 0) 산출물 파일 경로 ====
results_jsonl = ART_DIR / "results.jsonl"
manifest_path = ART_DIR / "manifest.json"
coverage_json_path = ART_DIR / "coverage_gen.json"     # 이번 라운드 통합(프루닝기존+생성)
coverage_xml_path  = ART_DIR / "coverage_gen.xml"
coverage_base_json = ART_DIR / "coverage_base.json"    # 3-1 기준선(원본 기존 테스트)

for old in [results_jsonl, coverage_json_path, coverage_xml_path]:
    if old.exists():
        old.unlink()

runs = []
ok = fail = to_cnt = 0

# ==== 1) (프루닝된) 기존 테스트 개별 격리 실행 ====
base_root = PRUNED_DIR if PRUNED_DIR.exists() else TESTS_DIR
base_files = list_test_files(base_root)

if base_files:
    print(f"🧪 프루닝된 기존 테스트 파일: {len(base_files)}개 @ {rel_to_proj(base_root)}")
    for tf in base_files:
        name = tf.name
        shard = COV_SHARDS_DIR / f".coverage.__base__.{name}"

        env = ENV_BASE.copy()
        env["PYTHONPATH"] = f"{PROJ}:{env.get('PYTHONPATH','')}"
        env["COVERAGE_FILE"] = str(shard)
        env.setdefault("NO_PROXY", "*")

        target = rel_to_proj(tf)
        cmd = f"{PY_EXE} -m coverage run{rc_opt} -m pytest {PYTEST_FLAGS} {shlex.quote(target)}"

        start = time.time()
        ts_start = datetime.now(timezone.utc).isoformat()
        rc, out, err, timed_out = sh(cmd, cwd=PROJ, timeout=TIMEOUT_SEC_BASE, env=env)
        dur = round(time.time() - start, 3)
        ts_end = datetime.now(timezone.utc).isoformat()

        (LOG_DIR / f"__base__{name}.out.txt").write_text(out, encoding="utf-8")
        (LOG_DIR / f"__base__{name}.err.txt").write_text(err, encoding="utf-8")

        runs.append({
            "suite": "BASE",
            "test_file": name,
            "goal_id": None,
            "start_utc": ts_start,
            "end_utc": ts_end,
            "duration_sec": dur,
            "returncode": rc,
            "timed_out": timed_out,
            "stdout_len": len(out),
            "stderr_len": len(err),
            "shard_path": str(shard),
            "invoked_path": target,
        })

        if timed_out:
            to_cnt += 1
            print(f"⏱️ TIMEOUT [BASE] {name} ({dur}s)")
        elif rc == 0:
            ok += 1
            print(f"✅ PASS   [BASE] {name} ({dur}s)")
        else:
            fail += 1
            first_err = (err.strip().splitlines() or [''])[0]
            print(f"❌ FAIL   [BASE] {name} (rc={rc}, {dur}s) :: {first_err}")
else:
    print("ℹ️ 프루닝된 기존 테스트가 없어 BASE 실행을 건너뜁니다. (Pruned_Base_Tests/ 또는 tests/ 비어있음)")

# ==== 2) 생성 테스트 파일 개별 격리 실행 ====
test_files = list_generated_test_files()
print(f"🧪 생성 테스트 파일: {len(test_files)}개 @ {rel_to_proj(GEN_DIR)}")
for tf in test_files:
    name = tf.name
    gid = goal_id_from_name(name) or "----"
    shard = COV_SHARDS_DIR / f".coverage.{name}"

    env = ENV_BASE.copy()
    env["PYTHONPATH"] = f"{PROJ}:{env.get('PYTHONPATH','')}"
    env["COVERAGE_FILE"] = str(shard)
    env.setdefault("NO_PROXY", "*")

    target = rel_to_proj(tf)   # e.g. "generated_tests/test_0001_...py"
    cmd = f"{PY_EXE} -m coverage run{rc_opt} -m pytest {PYTEST_FLAGS} {shlex.quote(target)}"

    start = time.time()
    ts_start = datetime.now(timezone.utc).isoformat()
    rc, out, err, timed_out = sh(cmd, cwd=PROJ, timeout=TIMEOUT_SEC_GEN, env=env)
    dur = round(time.time() - start, 3)
    ts_end = datetime.now(timezone.utc).isoformat()

    (LOG_DIR / f"{name}.out.txt").write_text(out, encoding="utf-8")
    (LOG_DIR / f"{name}.err.txt").write_text(err, encoding="utf-8")

    runs.append({
        "suite": "GEN",
        "test_file": name,
        "goal_id": gid,
        "start_utc": ts_start,
        "end_utc": ts_end,
        "duration_sec": dur,
        "returncode": rc,
        "timed_out": timed_out,
        "stdout_len": len(out),
        "stderr_len": len(err),
        "shard_path": str(shard),
        "invoked_path": target,
    })

    if timed_out:
        to_cnt += 1
        print(f"⏱️ TIMEOUT [GEN] {name} ({dur}s)")
    elif rc == 0:
        ok += 1
        print(f"✅ PASS   [GEN] {name} ({dur}s)")
    else:
        fail += 1
        first_err = (err.strip().splitlines() or [''])[0]
        print(f"❌ FAIL   [GEN] {name} (rc={rc}, {dur}s) :: {first_err}")

# 실행 기록 저장
results_jsonl.write_text(
    "\n".join(json.dumps(r, ensure_ascii=False) for r in runs) + ("\n" if runs else ""),
    encoding="utf-8"
)

manifest = {
    "generated_at_utc": datetime.now(timezone.utc).isoformat(),
    "project": str(PROJ),
    "run_dir": str(ART_DIR),
    "tests_total": len([r for r in runs if r['suite'] in {'BASE','GEN'}]),
    "pass": ok,
    "fail": fail,
    "timeout": to_cnt,
    "logs_dir": str(LOG_DIR),
    "cov_shards_dir": str(COV_SHARDS_DIR),
    "base_root": str(rel_to_proj(base_root)) if base_files else None,
    "gen_root": str(rel_to_proj(GEN_DIR)),
}
manifest_path.write_text(json.dumps(manifest, ensure_ascii=False, indent=2), encoding="utf-8")
print("\n📦 실행 요약:", json.dumps({"pass": ok, "fail": fail, "timeout": to_cnt, "total": manifest['tests_total']}, ensure_ascii=False))

# ==== 3) 커버리지 결합(JSON/XML/HTML) ====
shards = sorted([p for p in COV_SHARDS_DIR.iterdir() if p.name.startswith(".coverage.")])
if not shards:
    print("⚠️ 커버리지 샤드가 없습니다. 테스트가 즉시 실패했을 수 있습니다.")
else:
    subprocess.call(f"coverage erase{rc_opt}", shell=True, cwd=str(PROJ))
    combine_cmd = "coverage combine" + rc_opt + " " + " ".join(shlex.quote(str(p)) for p in shards)
    print("> ", combine_cmd)
    subprocess.call(combine_cmd, shell=True, cwd=str(PROJ))
    subprocess.call(f"coverage json -o {coverage_json_path.name}{rc_opt}", shell=True, cwd=str(PROJ))
    subprocess.call(f"coverage xml  -o {coverage_xml_path.name}{rc_opt}",  shell=True, cwd=str(PROJ))
    subprocess.call(f"coverage html -d {HTML_DIR_GEN.name}{rc_opt}",       shell=True, cwd=str(PROJ))

    # 결과 파일을 run_artifacts에 복사 보관
    src_json = PROJ / coverage_json_path.name
    src_xml  = PROJ / coverage_xml_path.name
    if src_json.exists(): shutil.copy2(src_json, coverage_json_path)
    if src_xml.exists():  shutil.copy2(src_xml,  coverage_xml_path)

    print("✅ 커버리지 결합 완료")
    print(" - JSON :", coverage_json_path)
    print(" - XML  :", coverage_xml_path)
    print(" - HTML :", HTML_DIR_GEN / "index.html")

# ==== 4) 분기 관측/목표 달성률 계산 ====
observed_outcomes_gen = {}
branch_points = full_hit = half_hit = zero_hit = 0

if coverage_xml_path.exists():
    try:
        xml_root = etree.parse(str(coverage_xml_path)).getroot()
        for cls in xml_root.findall(".//class"):
            filename = cls.get("filename") or ""
            if not filename:
                continue
            abs_path = (PROJ / filename).resolve() if not Path(filename).is_absolute() else Path(filename)
            for line in cls.findall("./lines/line"):
                if line.get("branch") != "true":
                    continue
                try:
                    num = int(line.get("number"))
                except Exception:
                    continue
                cond = line.get("condition-coverage")  # "50% (1/2)"
                covered = total = 0
                if cond:
                    m = re.search(r"\((\d+)\s*/\s*(\d+)\)", cond)
                    if m:
                        covered, total = int(m.group(1)), int(m.group(2))
                if total == 0:
                    continue
                observed_outcomes_gen.setdefault(str(abs_path), {})[num] = {
                    "covered": covered, "total": total, "ratio": round(covered/total, 3)
                }
                branch_points += 1
                if covered == 0: zero_hit += 1
                elif covered == total: full_hit += 1
                else: half_hit += 1
    except Exception as e:
        print("⚠️ coverage_xml 파싱 실패:", e)

(ART_DIR / "observed_outcomes_gen.json").write_text(
    json.dumps(observed_outcomes_gen, ensure_ascii=False, indent=2),
    encoding="utf-8"
)

print(f"🧮 분기 관측 요약 → total:{branch_points}, full:{full_hit}, half:{half_hit}, zero:{zero_hit}")

# ==== 5) 목표 달성률(기본) ====
GOALS_FILE = ART_DIR / "goals_ranked.json"
if GOALS_FILE.exists() and coverage_json_path.exists():
    cov_json = json.loads((coverage_json_path).read_text(encoding="utf-8"))
    files_map = cov_json.get("files", {}) or {}

    def line_hit(fpath: str, ln: int) -> bool:
        finfo = files_map.get(fpath) or files_map.get(str(Path(fpath).resolve()))
        if not finfo:
            return False
        executed = set(finfo.get("executed_lines", []) or [])
        return ln in executed

    goals = json.loads(GOALS_FILE.read_text(encoding="utf-8"))
    goal_stats = []
    for g in goals:
        f = g["file"]
        abs1 = str((PROJ / f).resolve())
        abs2 = f
        hit = sum(1 for ln in g.get("target_lines", []) if line_hit(abs1, ln) or line_hit(abs2, ln))
        total = len(g.get("target_lines", [])) or 1
        goal_stats.append({"id": g["id"], "hit": hit, "total": total, "rate": round(hit/total, 3)})

    (ART_DIR / "goal_achievements.json").write_text(
        json.dumps(goal_stats, ensure_ascii=False, indent=2),
        encoding="utf-8"
    )
    hit_goals = sum(1 for s in goal_stats if s["hit"] > 0)
    print(f"🎯 목표 달성률: {hit_goals}/{len(goal_stats)} 목표가 ≥1 라인 도달")

# ==== 6) 베이스라인 대비 향상치(delta) 계산 ====
def load_json(p: Path, default=None):
    try:
        return json.loads(p.read_text(encoding="utf-8"))
    except Exception:
        return default

base = load_json(coverage_base_json, {"files": {}}) or {"files": {}}
gen  = load_json(coverage_json_path, {"files": {}}) or {"files": {}}
base_files = base.get("files", {}) or {}
gen_files  = gen.get("files", {})  or {}

def _sum_len(key, d):
    return sum(len((d.get(f, {}) or {}).get(key, []) or []) for f in d.keys())

base_exec = _sum_len("executed_lines", base_files)
base_miss = _sum_len("missing_lines",  base_files)
gen_exec  = _sum_len("executed_lines", gen_files)
gen_miss  = _sum_len("missing_lines",  gen_files)

delta = {
    "executed_lines_delta": gen_exec - base_exec,
    "missing_lines_delta":  base_miss - gen_miss,   # +면 미싱 감소
    "base_executed": base_exec,
    "gen_executed":  gen_exec,
    "base_missing":  base_miss,
    "gen_missing":   gen_miss,
}
(ART_DIR / "coverage_delta.json").write_text(json.dumps(delta, ensure_ascii=False, indent=2), encoding="utf-8")
print("📈 베이스라인 대비 향상치:", json.dumps(delta, ensure_ascii=False))

print("✅ 3-5 완료: (프루닝기존+생성) autosetup + 격리 실행/샤드 결합/분기·목표·향상치 산출")


✅ pytest.ini ensured → /content/money-transfer-project-template-python/pytest.ini
✅ conftest.py created → /content/money-transfer-project-template-python/generated_tests/conftest.py
🧪 프루닝된 기존 테스트 파일: 1개 @ Pruned_Base_Tests
✅ PASS   [BASE] test_run_worker.py (7.151s)
🧪 생성 테스트 파일: 3개 @ generated_tests
❌ FAIL   [GEN] test_gen_goal_1_1.py (rc=2, 6.723s) :: /usr/local/lib/python3.12/dist-packages/coverage/control.py:894: CoverageWarning: No data was collected. (no-data-collected)
❌ FAIL   [GEN] test_gen_goal_4_1.py (rc=1, 6.718s) :: /usr/local/lib/python3.12/dist-packages/coverage/control.py:894: CoverageWarning: No data was collected. (no-data-collected)
❌ FAIL   [GEN] test_gen_goal_4_2.py (rc=1, 6.76s) :: /usr/local/lib/python3.12/dist-packages/coverage/control.py:894: CoverageWarning: No data was collected. (no-data-collected)

📦 실행 요약: {"pass": 1, "fail": 3, "timeout": 0, "total": 4}
>  coverage combine --rcfile /content/money-transfer-project-template-python/.coveragerc /content/money-

In [12]:
#@title 3-6) 미커버라인 기반 Adaptive Refinement Round (NO goals version)

import os, sys, json, re
from pathlib import Path
from datetime import datetime, timezone

# ===== Paths =====
assert 'PROJ' in globals(), "3-0 먼저 실행하세요."
PROJ = Path(PROJ).resolve()
ART_DIR = PROJ / "run_artifacts" / "run1"
GEN_DIR = PROJ / "generated_tests"
LOG_DIR = ART_DIR / "logs"
COV_GEN = ART_DIR / "coverage_gen.json"

REFINE_DIR = ART_DIR / f"refine_round1"
REFINE_DIR.mkdir(parents=True, exist_ok=True)
REFINE_PROMPTS = REFINE_DIR / "llm_refine_prompts.jsonl"
REFINE_SUMMARY = REFINE_DIR / "refine_summary.json"

def load_json(path):
    try: return json.loads(path.read_text(encoding="utf-8"))
    except: return None

cov = load_json(COV_GEN)
if not cov:
    raise SystemExit("coverage_gen.json이 없습니다. 3-5 먼저 실행하세요.")

files_map = cov.get("files", {}) or {}

# ====== 미커버 라인 기반 target mapping ======
file_targets = {}
for fpath, finfo in files_map.items():
    miss = finfo.get("missing_lines", []) or []
    if miss:
        file_targets[fpath] = sorted(miss)

# ====== 생성된 테스트 목록 ======
gen_tests = [p for p in GEN_DIR.glob("*.py") if p.name != "conftest.py"]

if not gen_tests:
    print("생성된 테스트가 없어 보강 라운드를 종료합니다.")
    raise SystemExit(0)

# ====== 실패 테스트 감지 ======
def load_log_head(name):
    out = (LOG_DIR / f"{name}.out.txt")
    err = (LOG_DIR / f"{name}.err.txt")
    o = out.read_text(encoding="utf-8") if out.exists() else ""
    e = err.read_text(encoding="utf-8") if err.exists() else ""
    return (o[:1500], e[:1500])

def classify_failure(std, err):
    blob = std + "\n" + err
    if "AssertionError" in blob: return "assert"
    if "ImportError" in blob or "ModuleNotFoundError" in blob: return "import"
    if "TimeoutExpired" in blob or "timed out" in blob: return "timeout"
    if "SyntaxError" in blob: return "syntax"
    if "TypeError" in blob: return "type"
    return None

selected = []

for tfile in gen_tests:
    name = tfile.name
    code = tfile.read_text(encoding="utf-8")

    # 실패 여부
    out_h, err_h = load_log_head(name)
    ftype = classify_failure(out_h, err_h)

    # 어떤 파일을 importlib로 로드하는지 추출
    imods = re.findall(r"importlib\.import_module\(['\"]([^'\"]+)['\"]\)", code)

    hit_any = False
    target_info = []

    for mod in imods:
        for fpath, miss_lines in file_targets.items():
            # 해당 테스트가 다루는 파일이라고 가정
            target_info.append({
                "file": fpath,
                "miss_lines": miss_lines
            })
            if miss_lines:
                hit_any = False

    # 실패한 테스트는 항상 보강 후보로 포함
    if ftype or target_info:
        selected.append({
            "id": name,
            "filename": name,
            "failure": ftype,
            "stdout": out_h,
            "stderr": err_h,
            "original_code": code,
            "targets": target_info,
        })

# ===== 선택 없으면 종료 =====
if not selected:
    REFINE_PROMPTS.write_text("", encoding="utf-8")
    REFINE_SUMMARY.write_text(json.dumps({
        "round_dir": REFINE_DIR.name,
        "selected": 0,
        "reason": "미커버 기반으로 보강할 테스트 없음"
    }, ensure_ascii=False, indent=2), encoding="utf-8")

    print("보강할 테스트가 없습니다.")
    raise SystemExit(0)

# ====== LLM 프롬프트 생성 ======
SYSTEM = (
"당신은 pytest 테스트 파일을 보강해 미커버 라인을 실행하게 만드는 전문가입니다.\n"
"출력은 마크다운 없이 JSON 객체 하나입니다.\n"
"{\n"
'  "edits": [ {"id": "파일명", "new_code": "보강된 전체 테스트 코드"} ]\n'
"}\n"
"규칙:\n"
"• test 파일 전체를 new_code에 넣되 설명/주석 금지.\n"
"• importlib + getattr 방식 유지.\n"
"• pytest.raises 또는 assert 반드시 포함.\n"
"• 미커버 라인(target_lines)을 실제로 실행하게 입력값/흐름을 조정.\n"
"• 외부 I/O/net/time/Temporal 사용 금지, monkeypatch 허용.\n"
)

with REFINE_PROMPTS.open("w", encoding="utf-8") as fo:
    for rec in selected:
        user = {
            "schema_version": "refine-simple-v1",
            "test_id": rec["id"],
            "filename": rec["filename"],
            "failure": rec["failure"],
            "targets": rec["targets"],
            "stdout_head": rec["stdout"],
            "stderr_head": rec["stderr"],
            "original_code": rec["original_code"],
            "instructions": [
                "target_lines를 반드시 실행할 것.",
                "불필요한 skip 제거.",
                "pytest 경계값 테스트를 사용해 조건을 만족시킬 것.",
            ]
        }

        fo.write(json.dumps({
            "meta": {"filename": rec["filename"]},
            "messages": [
                {"role": "system", "content": SYSTEM},
                {"role": "user", "content": json.dumps(user, ensure_ascii=False, indent=2)}
            ]
        }, ensure_ascii=False) + "\n")

REFINE_SUMMARY.write_text(json.dumps({
    "round_dir": REFINE_DIR.name,
    "selected": len(selected),
    "tests": [s["filename"] for s in selected],
}, ensure_ascii=False, indent=2), encoding="utf-8")

print("✅ 미커버 기반 보강 라운드 준비 완료")
print(" - 선택된 테스트:", len(selected))
print(" - 프롬프트:", REFINE_PROMPTS)


✅ 미커버 기반 보강 라운드 준비 완료
 - 선택된 테스트: 3
 - 프롬프트: /content/money-transfer-project-template-python/run_artifacts/run1/refine_round1/llm_refine_prompts.jsonl


In [13]:
#@title 3-7) 미커버라인 기반 보강 적용기 (NO goals, coverage-only version)

import os, re, json, ast, shutil, time, subprocess, shlex, sys
from pathlib import Path
from datetime import datetime, timezone
import httpx
import backoff
from lxml import etree
from openai import OpenAI, APIError, RateLimitError, APIConnectionError

# ===== 기본 설정 =====
MODEL = "gpt-4o"
TIMEOUT_SINGLE = 30
MIN_EXEC_GAIN = 3          # 실행 라인 증가 최소 조건
PYTEST_FLAGS = "-q -s"

# ===== 경로 =====
assert 'PROJ' in globals(), "3-0 먼저 실행하세요."
PROJ = Path(PROJ).resolve()
ART_DIR = PROJ / "run_artifacts" / "run1"
GEN_DIR = PROJ / "generated_tests"

REFINE_DIRS = sorted([p for p in ART_DIR.iterdir() if p.name.startswith("refine_round")])
if not REFINE_DIRS:
    raise SystemExit("refine_roundN 폴더가 없습니다. 3-6 먼저 실행하세요.")
REFINE_DIR = REFINE_DIRS[-1]

PROMPTS = REFINE_DIR / "llm_refine_prompts.jsonl"
if not PROMPTS.exists() or PROMPTS.stat().st_size == 0:
    print("보강 프롬프트 없음. 종료.")
    raise SystemExit(0)

RAW_DIR = REFINE_DIR / "_raw_edits"
ERR_DIR = REFINE_DIR / "_errors"
STAGE_DIR = REFINE_DIR / "_staging"
ARCHIVE_DIR = REFINE_DIR / "_archive"
REJECT_DIR = REFINE_DIR / "_rejected"

for d in [RAW_DIR, ERR_DIR, STAGE_DIR, ARCHIVE_DIR, REJECT_DIR]:
    d.mkdir(parents=True, exist_ok=True)

# ===== API 클라이언트 =====
if not os.getenv("OPENAI_API_KEY"):
    raise RuntimeError("OPENAI_API_KEY missing")

http_client = httpx.Client(
    timeout=180, follow_redirects=True,
    limits=httpx.Limits(max_connections=1, max_keepalive_connections=0),
    transport=httpx.HTTPTransport(retries=5),
)

client = OpenAI(api_key=os.getenv("OPENAI_API_KEY"), http_client=http_client)

# ===== 기본 유틸 =====
def strip_fences(s):
    s = re.sub(r"^```[a-zA-Z0-9]*", "", (s or "").strip())
    s = re.sub(r"```$", "", s)
    return s

def sanitize_test_code(code):
    patterns = [
        re.compile(r"(?ms)^\s*if __name__ == ['\"]__main__['\"]:\s*\n.+$"),
        re.compile(r"^\s*pytest\.main\(.*?\)$", re.M),
        re.compile(r"^\s*unittest\.main\(.*?\)$", re.M),
    ]
    out = code
    for p in patterns:
        out = p.sub("", out)
    return out.strip() + "\n"

# ===== Coverage 단일 실행 =====
def run_test_with_coverage(path: Path):
    shard = STAGE_DIR / f".coverage_{path.name}"
    out_json = STAGE_DIR / f"{path.stem}.json"
    out_xml = STAGE_DIR / f"{path.stem}.xml"

    for f in [shard, out_json, out_xml]:
        if f.exists():
            f.unlink()

    env = os.environ.copy()
    env["PYTHONPATH"] = f"{PROJ}:{env.get('PYTHONPATH','')}"
    env["COVERAGE_FILE"] = str(shard)

    cmd = f"{sys.executable} -m coverage run -m pytest {PYTEST_FLAGS} {shlex.quote(str(path))}"

    try:
        rc = subprocess.run(
            cmd, shell=True, cwd=PROJ,
            stdout=subprocess.PIPE, stderr=subprocess.PIPE,
            timeout=TIMEOUT_SINGLE, text=True, env=env
        )
    except subprocess.TimeoutExpired:
        return {"rc": 124, "missing": {}, "exec": 0, "branch_cov": (0,0)}

    # collect coverage
    subprocess.run(f"coverage json -o {out_json}", shell=True, cwd=PROJ)
    subprocess.run(f"coverage xml -o {out_xml}", shell=True, cwd=PROJ)

    try:
        cov = json.loads(out_json.read_text(encoding="utf-8"))
        files = cov.get("files", {})
    except:
        return {"rc": rc.returncode, "missing": {}, "exec": 0, "branch_cov": (0,0)}

    # missing lines
    missing_map = {f: set(v.get("missing_lines", [])) for f, v in files.items()}
    exec_count = sum(len(v.get("executed_lines",[])) for v in files.values())

    # branch
    covered = total = 0
    try:
        root = etree.parse(str(out_xml)).getroot()
        for line in root.findall(".//line[@branch='true']"):
            m = re.search(r"\((\d+)\s*/\s*(\d+)\)", line.get("condition-coverage") or "")
            if m:
                covered += int(m.group(1))
                total += int(m.group(2))
    except:
        pass

    return {"rc": rc.returncode, "missing": missing_map,
            "exec": exec_count, "branch_cov": (covered, total)}

# ====== 개선 여부 판단 ======
def improved(base, cand, target_lines: set):
    if cand["rc"] != 0:
        return False

    # 1) missing 감소
    miss_base = len(target_lines & base["missing"])
    miss_cand = len(target_lines & cand["missing"])
    if miss_cand < miss_base:
        return True

    # 2) 실행 라인 증가
    if cand["exec"] >= base["exec"] + MIN_EXEC_GAIN:
        return True

    # 3) branch hit 증가
    bc_b, bt_b = base["branch_cov"]
    bc_c, bt_c = cand["branch_cov"]
    if bc_c > bc_b:
        return True

    return False

# ===== Main =====
LOG = []
with PROMPTS.open("r", encoding="utf-8") as f:
    for line in f:
        rec = json.loads(line)
        user_payload = json.loads(rec["messages"][1]["content"])

        filename = user_payload["filename"]
        targets = user_payload["targets"]
        test_orig = GEN_DIR / filename

        orig_code = test_orig.read_text(encoding="utf-8") if test_orig.exists() else ""

        # base metrics
        base_m = run_test_with_coverage(test_orig) if orig_code else {
            "rc": 1, "missing": {}, "exec": 0, "branch_cov": (0,0)
        }

        # call LLM
        try:
            out_raw = client.chat.completions.create(
                model=MODEL,
                messages=rec["messages"],
                response_format={"type": "json_object"},
            ).choices[0].message.content
        except Exception as e:
            ERR_DIR.joinpath(f"{filename}_request.json").write_text(
                json.dumps({"error": str(e)}, ensure_ascii=False, indent=2), encoding="utf-8")
            continue

        RAW_DIR.joinpath(f"{filename}_raw.json").write_text(out_raw, encoding="utf-8")

        try:
            result = json.loads(strip_fences(out_raw))
            edit = result["edits"][0]
            new_code = sanitize_test_code(strip_fences(edit["new_code"]))
        except Exception as e:
            ERR_DIR.joinpath(f"{filename}_parse.json").write_text(
                json.dumps({"error": str(e), "raw": out_raw[:2000]}, ensure_ascii=False, indent=2),
                encoding="utf-8")
            continue

        # 임시 저장 → 평가
        stage_file = STAGE_DIR / filename
        stage_file.write_text(new_code, encoding="utf-8")

        cand_m = run_test_with_coverage(stage_file)

        # target_lines 계산
        target_lines = set()
        for t in targets:
            target_lines |= set(t["miss_lines"])

        # 개선 여부 판단
        if improved(base_m, cand_m, target_lines):
            # 수락 → 원본 백업 후 교체
            if test_orig.exists():
                shutil.copy2(test_orig, ARCHIVE_DIR / f"{filename}.{int(time.time())}.bak")
            test_orig.write_text(new_code, encoding="utf-8")
            LOG.append({"file": filename, "decision": "ACCEPT"})
            print(f"✅ ACCEPT: {filename}")
        else:
            REJECT_DIR.joinpath(filename).write_text(new_code, encoding="utf-8")
            LOG.append({"file": filename, "decision": "REJECT"})
            print(f"❌ REJECT: {filename}")

# 저장
(REFINE_DIR / "apply_log.jsonl").write_text(
    "\n".join(json.dumps(x, ensure_ascii=False) for x in LOG),
    encoding="utf-8"
)

print("\n✅ 미커버라인 기반 보강 적용 완료")


❌ REJECT: test_gen_goal_4_1.py
❌ REJECT: test_gen_goal_1_1.py
❌ REJECT: test_gen_goal_4_2.py

✅ 미커버라인 기반 보강 적용 완료


In [14]:
#@title 3-8) 프루닝기존 + (생성 + 보강) 전체 테스트 통합 실행 · 커버리지 결합 · 향상치 산출 (ACTIVE_MANIFEST 무시 버전)

import os, sys, json, re, time, subprocess, shutil, shlex
from pathlib import Path
from datetime import datetime, timezone
from lxml import etree

# ==== 경로/상수 ====
assert 'PROJ' in globals(), "3-0 단계를 먼저 실행하세요."
PROJ = Path(PROJ).resolve()
ART_DIR = PROJ / "run_artifacts" / "run1"
GEN_DIR = PROJ / "generated_tests"
PRUNED_DIR = PROJ / "Pruned_Base_Tests"
LOG_DIR = ART_DIR / "logs"
COV_SHARDS_DIR = ART_DIR / "cov_shards"
HTML_DIR_GEN = PROJ / "htmlcov_gen"
ACTIVE_MANIFEST = GEN_DIR / "ACTIVE_MANIFEST.json"

ART_DIR.mkdir(parents=True, exist_ok=True)
LOG_DIR.mkdir(parents=True, exist_ok=True)
COV_SHARDS_DIR.mkdir(parents=True, exist_ok=True)
HTML_DIR_GEN.mkdir(parents=True, exist_ok=True)
GEN_DIR.mkdir(parents=True, exist_ok=True)

RCFILE = PROJ / ".coveragerc"
rc_opt = f" --rcfile {RCFILE}" if RCFILE.exists() else ""
PY_EXE = sys.executable
TIMEOUT_SEC_GEN = 45
TIMEOUT_SEC_BASE = 60
PYTEST_FLAGS = "-q -s"
ENV_BASE = os.environ.copy()
ENV_BASE["PYTHONPATH"] = f"{PROJ}:{ENV_BASE.get('PYTHONPATH','')}"
ENV_BASE.setdefault("NO_PROXY", "*")

RE_GOAL = re.compile(r"(?:^|[_-])(?P<gid>\d{4})(?:[_-]|$)")

def goal_id_from_name(name: str) -> str | None:
    m = RE_GOAL.search(name)
    return m.group("gid") if m else None

def sh(cmd: str, cwd: Path|None=None, timeout: int|None=None, env: dict|None=None):
    try:
        p = subprocess.run(cmd, cwd=str(cwd or PROJ), env=env or ENV_BASE,
                           shell=True, stdout=subprocess.PIPE, stderr=subprocess.PIPE,
                           timeout=timeout, text=True)
        return p.returncode, p.stdout, p.stderr, False
    except subprocess.TimeoutExpired as e:
        return 124, e.stdout or "", e.stderr or "", True

def rel_to_proj(p: Path) -> str:
    try: return str(p.resolve().relative_to(PROJ))
    except Exception: return str(p.resolve())

# ==== 실행 대상 구성 ====
def list_pruned_tests() -> list[Path]:
    if not PRUNED_DIR.exists():
        return []
    return sorted([p for p in PRUNED_DIR.glob("test*.py") if p.is_file()])

def list_all_generated_tests() -> list[Path]:
    """ACTIVE_MANIFEST와 관계없이 generated_tests/*.py 전체를 실행."""
    return sorted([
        p for p in GEN_DIR.glob("test*.py")
        if p.is_file() and p.name != "conftest.py"
    ])

# ==== 0) 산출물 경로 ====
results_jsonl = ART_DIR / "results.jsonl"
coverage_json_path = ART_DIR / "coverage_gen.json"
coverage_xml_path  = ART_DIR / "coverage_gen.xml"
for old in [results_jsonl, coverage_json_path, coverage_xml_path]:
    if old.exists(): old.unlink()

runs = []
ok = fail = to_cnt = 0

# ==== 1) 프루닝된 기존 테스트 실행 ====
base_files = list_pruned_tests()
if base_files:
    print(f"🧪 프루닝된 기존 테스트 파일: {len(base_files)}개 @ Pruned_Base_Tests/")
    for tf in base_files:
        name = tf.name
        shard = COV_SHARDS_DIR / f".coverage.__base__.{name}"
        env = ENV_BASE.copy()
        env["COVERAGE_FILE"] = str(shard)
        target = rel_to_proj(tf)
        cmd = f"{PY_EXE} -m coverage run{rc_opt} -m pytest {PYTEST_FLAGS} {shlex.quote(target)}"
        start = time.time()
        rc, out, err, timed_out = sh(cmd, timeout=TIMEOUT_SEC_BASE, env=env)
        dur = round(time.time()-start,3)
        (LOG_DIR / f"__base__{name}.out.txt").write_text(out,encoding="utf-8")
        (LOG_DIR / f"__base__{name}.err.txt").write_text(err,encoding="utf-8")
        if timed_out:
            to_cnt+=1; print(f"⏱️ TIMEOUT [BASE] {name} ({dur}s)")
        elif rc==0:
            ok+=1; print(f"✅ PASS   [BASE] {name} ({dur}s)")
        else:
            fail+=1; first=(err.strip().splitlines() or [''])[0]
            print(f"❌ FAIL   [BASE] {name} (rc={rc}, {dur}s) :: {first}")
else:
    print("ℹ️ 프루닝된 기존 테스트가 없어 BASE 실행 건너뜀")

# ==== 2) 생성 + 보강 테스트 전체 실행 (ACTIVE_MANIFEST 무시) ====
gen_files = list_all_generated_tests()
print(f"🧪 생성/보강 테스트 파일: {len(gen_files)}개 @ generated_tests/ (ACTIVE_MANIFEST 무시)")

for tf in gen_files:
    name = tf.name
    gid = goal_id_from_name(name) or "----"
    shard = COV_SHARDS_DIR / f".coverage.{name}"
    env = ENV_BASE.copy()
    env["COVERAGE_FILE"] = str(shard)
    target = rel_to_proj(tf)
    cmd = f"{PY_EXE} -m coverage run{rc_opt} -m pytest {PYTEST_FLAGS} {shlex.quote(target)}"
    start = time.time()
    rc,out,err,timed_out = sh(cmd,timeout=TIMEOUT_SEC_GEN,env=env)
    dur = round(time.time()-start,3)
    (LOG_DIR / f"{name}.out.txt").write_text(out,encoding="utf-8")
    (LOG_DIR / f"{name}.err.txt").write_text(err,encoding="utf-8")
    if timed_out:
        to_cnt+=1; print(f"⏱️ TIMEOUT [GEN] {name} ({dur}s)")
    elif rc==0:
        ok+=1; print(f"✅ PASS   [GEN] {name} ({dur}s)")
    else:
        fail+=1; first=(err.strip().splitlines() or [''])[0]
        print(f"❌ FAIL   [GEN] {name} (rc={rc}, {dur}s) :: {first}")

# ==== 3) 커버리지 결합 ====
shards = [p for p in COV_SHARDS_DIR.iterdir() if p.name.startswith(".coverage.")]
if not shards:
    print("⚠️ 커버리지 샤드가 없습니다. (모든 테스트 실패 가능성)")
else:
    subprocess.call(f"coverage erase{rc_opt}",shell=True,cwd=str(PROJ))
    combine = "coverage combine"+rc_opt+" "+" ".join(shlex.quote(str(p)) for p in shards)
    print("> ",combine)
    subprocess.call(combine,shell=True,cwd=str(PROJ))
    subprocess.call(f"coverage json -o coverage_gen.json{rc_opt}",shell=True,cwd=str(PROJ))
    subprocess.call(f"coverage xml  -o coverage_gen.xml{rc_opt}",shell=True,cwd=str(PROJ))
    subprocess.call(f"coverage html -d {HTML_DIR_GEN.name}{rc_opt}",shell=True,cwd=str(PROJ))
    shutil.copy2(PROJ/"coverage_gen.json",coverage_json_path)
    shutil.copy2(PROJ/"coverage_gen.xml",coverage_xml_path)
    print("✅ 커버리지 결합 완료")
    print(" - JSON :",coverage_json_path)
    print(" - XML  :",coverage_xml_path)
    print(" - HTML :",HTML_DIR_GEN/"index.html")

# ==== 4) 목표 달성률 및 향상치 계산 ====
def load_json(p,default=None):
    try: return json.loads(p.read_text(encoding="utf-8"))
    except Exception: return default
base = load_json(ART_DIR/"coverage_base.json",{"files":{}}) or {"files":{}}
gen  = load_json(coverage_json_path,{"files":{}}) or {"files":{}}
def _sum_len(key,d): return sum(len((d.get(f, {}) or {}).get(key,[]) or []) for f in d)
base_exec=_sum_len("executed_lines",base["files"])
base_miss=_sum_len("missing_lines",base["files"])
gen_exec=_sum_len("executed_lines",gen["files"])
gen_miss=_sum_len("missing_lines",gen["files"])
delta={
    "executed_lines_delta":gen_exec-base_exec,
    "missing_lines_delta": base_miss-gen_miss,
    "base_executed":base_exec,"gen_executed":gen_exec,
    "base_missing":base_miss,"gen_missing":gen_miss
}
(ART_DIR/"coverage_delta.json").write_text(json.dumps(delta,ensure_ascii=False,indent=2),encoding="utf-8")
print("📈 베이스라인 대비 향상치:",json.dumps(delta,ensure_ascii=False))
print("✅ 3-8 완료: 전체 테스트 실행 + 커버리지 산출 (ACTIVE_MANIFEST 무시 버전)")


🧪 프루닝된 기존 테스트 파일: 1개 @ Pruned_Base_Tests/
✅ PASS   [BASE] test_run_worker.py (7.255s)
🧪 생성/보강 테스트 파일: 3개 @ generated_tests/ (ACTIVE_MANIFEST 무시)
❌ FAIL   [GEN] test_gen_goal_1_1.py (rc=2, 6.207s) :: /usr/local/lib/python3.12/dist-packages/coverage/control.py:894: CoverageWarning: No data was collected. (no-data-collected)
❌ FAIL   [GEN] test_gen_goal_4_1.py (rc=1, 7.565s) :: /usr/local/lib/python3.12/dist-packages/coverage/control.py:894: CoverageWarning: No data was collected. (no-data-collected)
❌ FAIL   [GEN] test_gen_goal_4_2.py (rc=1, 6.787s) :: /usr/local/lib/python3.12/dist-packages/coverage/control.py:894: CoverageWarning: No data was collected. (no-data-collected)
>  coverage combine --rcfile /content/money-transfer-project-template-python/.coveragerc /content/money-transfer-project-template-python/run_artifacts/run1/cov_shards/.coverage.test_gen_goal_4_2.py /content/money-transfer-project-template-python/run_artifacts/run1/cov_shards/.coverage.test_gen_goal_1_1.py /content/mo

In [15]:
#@title 3-9 최종 Test Suite 생성 (미커버라인 기반 전용 버전)

from pathlib import Path
import os, shutil, re

PROJ = Path(PROJ).resolve()
ART_DIR = PROJ / "run_artifacts" / "run1"
GEN_DIR = PROJ / "generated_tests"
PRUNED_DIR = PROJ / "Pruned_Base_Tests"
LOG_DIR = ART_DIR / "logs"

FINAL_SUITE = PROJ / "Final_Test_Suite"

# Reset
if FINAL_SUITE.exists():
    shutil.rmtree(FINAL_SUITE)
FINAL_SUITE.mkdir(parents=True, exist_ok=True)


def is_pass_log(text):
    text = text.strip().lower()

    if "fail" in text or "failed" in text:
        return False
    if "error" in text:
        return False

    # pytest -q 패턴
    if "passed" in text:
        return True

    # .ss.s. 스타일
    if re.fullmatch(r"[\.s]+", text):
        if "f" not in text and "e" not in text:
            return True

    return False


pass_generated = []

# 1) 생성·보강 테스트 PASS 판정
for out_path in LOG_DIR.glob("*.py.out.txt"):     # ★ 전체 test 파일 대상
    name = out_path.name.replace(".out.txt", "")
    if not name.startswith("test"):               # ★ 안전 필터
        continue

    err_path = LOG_DIR / f"{name}.err.txt"

    out_text = out_path.read_text(encoding="utf-8")
    err_text = err_path.read_text(encoding="utf-8") if err_path.exists() else ""

    # FAIL/ERROR anywhere → 제외
    if "fail" in out_text.lower() or "fail" in err_text.lower():
        continue
    if "error" in out_text.lower() or "error" in err_text.lower():
        continue

    if is_pass_log(out_text):
        if (GEN_DIR / name).exists():
            pass_generated.append(name)


# 2) 기존 프루닝 테스트 복사
base_tests = list(PRUNED_DIR.glob("test*.py"))
for tf in base_tests:
    shutil.copy2(tf, FINAL_SUITE / tf.name)


# 3) PASS 생성/보강 테스트 복사
for fname in pass_generated:
    shutil.copy2(GEN_DIR / fname, FINAL_SUITE / fname)

print("✅ FINAL TEST SUITE CONSTRUCTED")
print("기존 프루닝 테스트:", len(base_tests))
print("PASS 생성/보강 테스트:", len(pass_generated))
print("→ 최종 폴더:", FINAL_SUITE)


✅ FINAL TEST SUITE CONSTRUCTED
기존 프루닝 테스트: 1
PASS 생성/보강 테스트: 0
→ 최종 폴더: /content/money-transfer-project-template-python/Final_Test_Suite


In [18]:
#@title 3-10) 최종 성능 분석 – 테스트 함수 수 + 테스트별 커버 라인/함수 분석

import os, json, subprocess, shlex, re, sys
from pathlib import Path
from lxml import etree

PROJ = Path(PROJ).resolve()
ART_DIR = PROJ / "run_artifacts" / "run1"
FINAL_SUITE = PROJ / "Final_Test_Suite"
ORIG_TESTS = PROJ / "tests"

RCFILE = PROJ / ".coveragerc"
rc_opt = f" --rcfile {RCFILE}" if RCFILE.exists() else ""
PYTEST_FLAGS = "-q -s"
PY_EXE = sys.executable


# =====================================================
# 유틸 함수들
# =====================================================
def sh(cmd, cwd=None):
    p = subprocess.run(
        cmd, cwd=cwd or PROJ, shell=True,
        stdout=subprocess.PIPE, stderr=subprocess.PIPE,
        text=True
    )
    return p.returncode, p.stdout, p.stderr


def get_test_count(path):
    """pytest --collect-only 로 테스트 개수 정확히 수집"""
    rc, out, err = sh(f"{PY_EXE} -m pytest --collect-only {path}")
    text = out + err
    m = re.search(r"collected\s+(\d+)\s+items", text)
    return int(m.group(1)) if m else 0


def erase_cov():
    for nm in [".coverage", "coverage.json", "coverage.xml"]:
        p = PROJ / nm
        if p.exists():
            p.unlink()


def export_cov(json_name, xml_name):
    subprocess.call(f"coverage json -o {json_name}{rc_opt}", shell=True, cwd=str(PROJ))
    subprocess.call(f"coverage xml  -o {xml_name}{rc_opt}", shell=True, cwd=str(PROJ))


def load_cov_json(path):
    return json.load(open(PROJ / path, "r", encoding="utf-8"))


def parse_branch(xml_path):
    root = etree.parse(str(PROJ / xml_path)).getroot()
    hit = tot = 0
    for ln in root.findall(".//line[@branch='true']"):
        cc = ln.get("condition-coverage") or ""
        m = re.search(r"\((\d+)\s*/\s*(\d+)\)", cc)
        if m:
            hit += int(m.group(1))
            tot += int(m.group(2))
    return hit, tot


# =====================================================
# (수동 분모) 기준 분모 설정
# =====================================================
print("📌 기준 분모 측정 시작...")

TOTAL_LINES = 235
TOTAL_BRANCHES = 72

print(f"📌 기준 실행가능 라인 : {TOTAL_LINES}")
print(f"📌 기준 실행가능 브랜치 : {TOTAL_BRANCHES}")


# =====================================================
# 최초 테스트 실행
# =====================================================
print("\n📌 최초 tests/ 실행 중...")

orig_test_count = get_test_count("tests/")

erase_cov()
rc, out, err = sh(f"{PY_EXE} -m coverage run{rc_opt} -m pytest -q -s tests/")
export_cov("cov_orig.json", "cov_orig.xml")

cov_orig = load_cov_json("cov_orig.json")
orig_executed = sum(len(v.get("executed_lines", [])) for v in cov_orig["files"].values())
BR_O_HIT, BR_O_TOT = parse_branch("cov_orig.xml")


# =====================================================
# Final_Test_Suite 실행
# =====================================================
print("\n📌 Final_Test_Suite 실행 중...")

final_test_count = get_test_count(str(FINAL_SUITE))

erase_cov()
rc2, out2, err2 = sh(f"{PY_EXE} -m coverage run{rc_opt} -m pytest -q -s {FINAL_SUITE}")
export_cov("cov_final.json", "cov_final.xml")

cov_final = load_cov_json("cov_final.json")
final_executed = sum(len(v.get("executed_lines", [])) for v in cov_final["files"].values())
BR_F_HIT, BR_F_TOT = parse_branch("cov_final.xml")

BR_DEN = TOTAL_BRANCHES


# =====================================================
# 신규 라인 분석
# =====================================================
orig_lines = {
    f"{f}:{l}"
    for f, info in cov_orig["files"].items()
    for l in info.get("executed_lines", [])
}
final_lines = {
    f"{f}:{l}"
    for f, info in cov_final["files"].items()
    for l in info.get("executed_lines", [])
}

new_lines = sorted(list(final_lines - orig_lines))


# =====================================================
# 테스트별 라인/함수 분석
# =====================================================
print("\n📌 테스트별 커버리지 분석 중...")

test_cover_detail = {}

for fpath in cov_final["files"]:
    if "/generated_tests/" not in fpath and "/Final_Test_Suite/" not in fpath:
        continue

    info = cov_final["files"][fpath]
    executed = info.get("executed_lines", [])
    funcs = set()

    source_path = PROJ / fpath
    if source_path.exists():
        import ast
        tree = ast.parse(source_path.read_text(encoding="utf-8"))
        for node in ast.walk(tree):
            if isinstance(node, ast.FunctionDef):
                start = node.lineno
                end = max([n.lineno for n in ast.walk(node) if hasattr(n, "lineno")], default=start)
                if any(start <= l <= end for l in executed):
                    funcs.add(node.name)

    test_cover_detail[fpath] = {
        "executed_lines": executed,
        "covered_functions": sorted(list(funcs))
    }


# =====================================================
# 결과 출력
# =====================================================
print("\n📊 3-10 최종 분석 결과\n")

print("1️⃣ 실행된 테스트 함수 수")
print(f" - 최초 tests/: {orig_test_count}개")
print(f" - 최종 Final_Test_Suite: {final_test_count}개")
print(f" - Δ 테스트 증가: {final_test_count - orig_test_count}개\n")

print("2️⃣ 라인 커버리지 (기준 분모 대비)")
print(f" - 최초: {round(orig_executed/TOTAL_LINES*100,2)}% ({orig_executed}/{TOTAL_LINES})")
print(f" - 최종: {round(final_executed/TOTAL_LINES*100,2)}% ({final_executed}/{TOTAL_LINES})")
print(f" - Δ 라인 커버리지: {round((final_executed-orig_executed)/TOTAL_LINES*100,2)}%p\n")

print("3️⃣ 브랜치 커버리지 (기준 분모 대비)")
print(f" - 최초: {round(BR_O_HIT/BR_DEN*100,2)}% ({BR_O_HIT}/{BR_DEN})")
print(f" - 최종: {round(BR_F_HIT/BR_DEN*100,2)}% ({BR_F_HIT}/{BR_DEN})")
print(f" - Δ 브랜치 커버리지: {round((BR_F_HIT-BR_O_HIT)/BR_DEN*100,2)}%p\n")

print("4️⃣ 새롭게 실행된 소스코드 라인")
print(f" - 총 {len(new_lines)}개 증가")
for ln in new_lines[:40]:
    print("   ", ln)
if len(new_lines) > 40:
    print("   ... 생략 ...")

print("\n5️⃣ 테스트별 라인/함수 커버리지\n")
for f, detail in test_cover_detail.items():
    print(f"📄 {f}")
    print(f" - 커버 라인 수: {len(detail['executed_lines'])}")
    print(f" - 커버 함수: {detail['covered_functions']}\n")

print("\n✅ 3-10 전체 분석 완료 (수동 분모 버전)")


📌 기준 분모 측정 시작...
📌 기준 실행가능 라인 : 235
📌 기준 실행가능 브랜치 : 72

📌 최초 tests/ 실행 중...

📌 Final_Test_Suite 실행 중...

📌 테스트별 커버리지 분석 중...

📊 3-10 최종 분석 결과

1️⃣ 실행된 테스트 함수 수
 - 최초 tests/: 0개
 - 최종 Final_Test_Suite: 0개
 - Δ 테스트 증가: 0개

2️⃣ 라인 커버리지 (기준 분모 대비)
 - 최초: 43.4% (102/235)
 - 최종: 59.15% (139/235)
 - Δ 라인 커버리지: 15.74%p

3️⃣ 브랜치 커버리지 (기준 분모 대비)
 - 최초: 44.44% (32/72)
 - 최종: 75.0% (54/72)
 - Δ 브랜치 커버리지: 30.56%p

4️⃣ 새롭게 실행된 소스코드 라인
 - 총 37개 증가
    Final_Test_Suite/test_run_worker.py:10
    Final_Test_Suite/test_run_worker.py:11
    Final_Test_Suite/test_run_worker.py:14
    Final_Test_Suite/test_run_worker.py:15
    Final_Test_Suite/test_run_worker.py:16
    Final_Test_Suite/test_run_worker.py:17
    Final_Test_Suite/test_run_worker.py:18
    Final_Test_Suite/test_run_worker.py:2
    Final_Test_Suite/test_run_worker.py:24
    Final_Test_Suite/test_run_worker.py:25
    Final_Test_Suite/test_run_worker.py:31
    Final_Test_Suite/test_run_worker.py:37
    Final_Test_Suite/test_run_worker.py:4
    Fi

In [19]:
import shutil
import pathlib

proj = pathlib.Path(PROJ)
zip_path = proj / "My_Research_experiment_outputs_NoCompositeVer.zip"

# 압축 대상: run_artifacts, generated_tests, htmlcov, reports
targets = ["run_artifacts", "generated_tests", "reports", "htmlcov", "Final_Test_Suite"]

# 임시 디렉토리 구조를 zip에 담기 위해 새 폴더 구성
temp_root = proj / "_zip_bundle"
if temp_root.exists():
    shutil.rmtree(temp_root)
temp_root.mkdir()

for name in targets:
    src = proj / name
    if src.exists():
        shutil.copytree(src, temp_root / name)

# ZIP 생성
shutil.make_archive(str(zip_path.with_suffix("")), 'zip', temp_root)

print("압축 생성 완료:", zip_path)


압축 생성 완료: /content/money-transfer-project-template-python/My_Research_experiment_outputs_NoCompositeVer.zip
